In [ ]:
import pandas as pd 
import numpy as np
from matplotlib import rc
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, MaxNLocator
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import resample
import os
from scipy.stats import pearsonr
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pickle
import matplotlib_venn
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.lines as lines
import matplotlib.ticker as ticker
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles
import matplotlib.patheffects as path_effects
import matplotlib.transforms as transforms

In [ ]:
# Define a function for CA
def commonality_analysis(features_g_pred, target_g_real):
    if len(features_g_pred.shape)<2:
        r2 = LinearRegression().fit(features_g_pred.values.reshape(-1, 1), target_g_real).score(features_g_pred.values.reshape(-1, 1), target_g_real).round(4)
    else:
        r2 = LinearRegression().fit(features_g_pred, target_g_real).score(features_g_pred, target_g_real).round(4)
    return r2

# Main analysis: no demo, only brain and mental health

# RS Stacked

In [ ]:
# Get R2 for each model
folds = ['0','1','2','3','4']
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
all_g_mh_rs_concat = []
for fold in folds:
    g_pred_rs_idp_rf = pd.read_csv(f'/PLS/brain/stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_rs_idp_rf, on='eid').drop(columns=['eid'])
    all_g.columns = ['g_real', 'g_pred_mh', 'g_pred_rs_ts']
    all_g_mh_rs_concat.append(all_g)
    all_g_mh_rs = pd.concat(all_g_mh_rs_concat, axis=0, ignore_index=True)
    all_g_mh_rs.to_csv('/commonality_analysis/rs/g_real_pred_mh_rs-idp-atlases.csv', index=False)

model = LinearRegression()
r2_mh_rs = model.fit(all_g_mh_rs['g_pred_mh'].values.reshape(-1, 1), all_g_mh_rs['g_real']).score(all_g_mh_rs['g_pred_mh'].values.reshape(-1, 1), all_g_mh_rs['g_real'])
r2_rs = model.fit(all_g_mh_rs['g_pred_rs_ts'].values.reshape(-1, 1), all_g_mh_rs['g_real']).score(all_g_mh_rs['g_pred_rs_ts'].values.reshape(-1, 1), all_g_mh_rs['g_real'])
r2_mh_and_rs = model.fit(pd.concat([all_g_mh_rs['g_pred_rs_ts'], all_g_mh_rs['g_pred_mh']], axis=1), all_g_mh_rs['g_real']).score(pd.concat([all_g_mh_rs['g_pred_rs_ts'], all_g_mh_rs['g_pred_mh']], axis=1), all_g_mh_rs['g_real'])

#### Run CA

In [ ]:
all_g_mh_rs = pd.read_csv('/commonality_analysis/rs/g_real_pred_mh_rs-idp-atlases.csv')
r2_mh_rs = commonality_analysis(all_g_mh_rs['g_pred_mh'], all_g_mh_rs['g_real'])
r2_rs = commonality_analysis(all_g_mh_rs['g_pred_rs_ts'], all_g_mh_rs['g_real'])
r2_mh_and_rs = commonality_analysis(pd.concat([all_g_mh_rs['g_pred_rs_ts'], all_g_mh_rs['g_pred_mh']], axis=1), all_g_mh_rs['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

u_mh_rs = r2_mh_and_rs - r2_rs
u_rs = r2_mh_and_rs - r2_mh_rs

c_mh_and_rs = r2_mh_and_rs - u_mh_rs - u_rs

print("R squared for MH:", r2_mh_rs.round(3)) # f"{r2_mh_rs_m:.5f}")
print("R squared for RS", r2_rs.round(3))
print("R squared for MH and RS", r2_mh_and_rs.round(3))

print('_______________')
print("Unique variance for MH:", u_mh_rs.round(3)) #"{:.5f}".format(u_mh_rs)) # "{:.5f}".format(u_mh_rs)
print("Unique variance for RS:", u_rs.round(3)) #f"{u_rs:.5f}")

print('_______________')

print("Common variance for MH and RS:", c_mh_and_rs.round(3))
print('_______________')
perc_mh = u_mh_rs / r2_mh_and_rs * 100 #u_mh_rs / R squared for MH and RS
perc_rs = u_rs / r2_mh_and_rs * 100 #u_rs / R squared for MH and RS
perc_c = c_mh_and_rs / r2_mh_and_rs * 100 #c_mh_and_rs / R squared for MH and RS

print(f"Proportion of unique variance attributed to MH is {perc_mh.round(2)}")
print(f"Proportion of unique variance attributed to Resting state (atlases) is {perc_rs.round(2)}")
print(f"Proportion of common variance attributed to MH and Resting state (atlases) is {perc_c.round(2)}")

# RS PLS

In [ ]:
# 1 Match g: original, from mental health, and from resting state
folds = [f'{i}' for i in range(5)]
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pls_g_mh_rs_concat = []
for fold in folds:
    g_pred_pls_test_rs = pd.read_csv(f'/PLS/brain/stacking/features_test_level1_stacked/rs_idp_ts_best_metrics/features_test_level1_g_matched_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    pls_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_pls_test_rs, on='eid').drop(columns=['eid'])
    pls_g_mh_rs_concat.append(pls_g)
    pls_g_mh_rs = pd.concat(pls_g_mh_rs_concat, axis=0, ignore_index=True)
    pls_g_mh_rs.to_csv('/commonality_analysis/rs/pls/g_real_pred_mh_rs-pls.csv', index=False)

In [ ]:
# 2 Extract R2 with commonality_analysis function
pls_g_mh_rs = pd.read_csv('/commonality_analysis/rs/pls/g_real_pred_mh_rs-pls.csv')
r2_mh_from_rs_pls = LinearRegression().fit(pls_g_mh_rs['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_rs['g']).score(pls_g_mh_rs['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_rs['g'])

rs_cols= [
    'aparc_s1_full_correlation', 'aparc_2009_s1_full_correlation', 
    'glasser_s1_full_correlation', 'glasser_s4_full_correlation', 
    'Schaefer7n200p_s1_full_correlation', 'Schaefer7n500p_s4_full_correlation', 
    'amplitudes_21', 'amplitudes_55', 'tangent_matrices_21', 'tangent_matrices_55'
]
r2_rs = {}
for col in rs_cols:
    r2_score = commonality_analysis(pls_g_mh_rs[col], pls_g_mh_rs['g'])
    r2_rs[col] = r2_score
    globals()[f'r2_{col}'] = r2_score

# Iterate through each column to combine with 'g_pred_mh'
r2_mh_and_rs = {}
for col in rs_cols:
    combined_columns = pd.concat([pls_g_mh_rs[col], pls_g_mh_rs['g_pred_mh']], axis=1)
    r2_combined = commonality_analysis(combined_columns, pls_g_mh_rs['g'])
    r2_mh_and_rs[f'r2_mh_and_{col}'] = r2_combined
    # Create a variable for each combined R^2 score
    globals()[f'r2_mh_and_{col}'] = r2_combined

In [ ]:
# Save R2 dictionaries
with open('/commonality_analysis/rs/pls/r2_rs.pkl', 'wb') as f:
    pickle.dump(r2_rs, f)

with open('/commonality_analysis/rs/pls/r2_mh_and_rs.pkl', 'wb') as f:
    pickle.dump(r2_mh_and_rs, f)

In [ ]:
# 3 Calculate  unique and common variances
unique_mh = {}
unique_rs = {}
common_mh_rs = {}

for key in r2_mh_and_rs:
    rs_key = key.replace('r2_mh_and_', '')
    u_mh = r2_mh_and_rs[key] - r2_rs[rs_key]
    u_rs = r2_mh_and_rs[key] - r2_mh_from_rs_pls #[key]
    c_mh_and_rs = r2_mh_and_rs[key] - u_mh - u_rs
    
    unique_mh[f'u_mh_from_{rs_key}'] = u_mh
    unique_rs[f'u_rs_{rs_key}'] = u_rs
    common_mh_rs[f'c_mh_and_rs_{rs_key}'] = c_mh_and_rs

# Print results
for key in r2_mh_and_rs:
    rs_key = key.replace('r2_mh_and_', '')
    print(f"R squared for {rs_key}: {r2_rs[rs_key]:.3f}")
    print(f"R squared for MH and {rs_key}: {r2_mh_and_rs[key]:.3f}")
    print(f"Unique variance for MH ({rs_key}): {unique_mh[f'u_mh_from_{rs_key}']:.3f}")
    print(f"Unique variance for RS ({rs_key}): {unique_rs[f'u_rs_{rs_key}']:.3f}")
    print(f"Common variance for MH and RS ({rs_key}): {common_mh_rs[f'c_mh_and_rs_{rs_key}']:.3f}")
    print('_______________')

# Calculate proportions
for key in r2_mh_and_rs:
    rs_key = key.replace('r2_mh_and_', '')
    perc_mh = unique_mh[f'u_mh_from_{rs_key}'] / r2_mh_and_rs[key] * 100
    perc_rs = unique_rs[f'u_rs_{rs_key}'] / r2_mh_and_rs[key] * 100
    perc_c = common_mh_rs[f'c_mh_and_rs_{rs_key}'] / r2_mh_and_rs[key] * 100
    
    print(f"Proportion of unique variance attributed to MH ({rs_key}): {perc_mh:.2f}%")
    print(f"Proportion of unique variance attributed to RS ({rs_key}): {perc_rs:.2f}%")
    print(f"Proportion of common variance attributed to MH and RS ({rs_key}): {perc_c:.2f}%")
    print('_______________')

In [ ]:
# Save unique and common variances
with open('/commonality_analysis/rs/pls/unique_mh.pkl', 'wb') as f:
    pickle.dump(unique_mh, f)

with open('/commonality_analysis/rs/pls/unique_rs.pkl', 'wb') as f:
    pickle.dump(unique_rs, f)

with open('/commonality_analysis/rs/pls/common_mh_rs.pkl', 'wb') as f:
    pickle.dump(common_mh_rs, f)

#### Upload CA variables for RS

In [ ]:
# R2
pls_g_mh_rs = pd.read_csv('/commonality_analysis/rs/pls/g_real_pred_mh_rs-pls.csv')
r2_mh_from_rs_pls = LinearRegression().fit(pls_g_mh_rs['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_rs['g']).score(pls_g_mh_rs['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_rs['g'])

# Load R2 dictionaries from pickle files
with open('/commonality_analysis/rs/pls/r2_rs.pkl', 'rb') as f:
    r2_rs = pickle.load(f)

with open('/commonality_analysis/rs/pls/r2_mh_and_rs.pkl', 'rb') as f:
    r2_mh_and_rs = pickle.load(f)

# Extract variables
for col in r2_rs:
    globals()[col] = round(r2_rs[col] * 100, 3)
    r2_rs[col] = round(r2_rs[col] * 100, 3)  # Update the dictionary with rounded values

for col in r2_mh_and_rs:
    globals()[f'r2_mh_and_{col}'] = round(r2_mh_and_rs[col] * 100, 3)
    r2_mh_and_rs[col] = round(r2_mh_and_rs[col] * 100, 3)

print(r2_rs)
print(r2_mh_and_rs)

In [ ]:
# U/C
with open('/commonality_analysis/rs/pls/unique_mh.pkl', 'rb') as f:
    unique_mh = pickle.load(f)

with open('/commonality_analysis/rs/pls/unique_rs.pkl', 'rb') as f:
    unique_rs = pickle.load(f)

with open('/commonality_analysis/rs/pls/common_mh_rs.pkl', 'rb') as f:
    common_mh_rs = pickle.load(f)

# Extract variables
for key in unique_mh:
    globals()[key] = round(unique_mh[key] * 100, 3)
    unique_mh[key] = round(unique_mh[key] * 100, 3)  # Update the dictionary with rounded values

for key in unique_rs:
    globals()[key] = round(unique_rs[key] * 100, 3)
    unique_rs[key] = round(unique_rs[key] * 100, 3) 

for key in common_mh_rs:
    globals()[key] = round(common_mh_rs[key] * 100, 3)
    common_mh_rs[key] = round(common_mh_rs[key] * 100, 3)

print(unique_mh)
print(unique_rs)
print(common_mh_rs)

##########################################################

# DTI Stacked

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
all_g_mh_dti_all_concat = []

for fold in folds:
    g_pred_dti_all_rf = pd.read_csv(f'/PLS/brain/stacking/g/DTI_All_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_dti_all_rf, on='eid').drop(columns=['eid'])
    all_g.columns = ['g_real', 'g_pred_mh', 'g_pred_dti_all']
    all_g_mh_dti_all_concat.append(all_g)
    all_g_mh_dti_all = pd.concat(all_g_mh_dti_all_concat, axis=0, ignore_index=True)
    all_g_mh_dti_all.to_csv('/commonality_analysis/dti/g_real_pred_mh_dti-all.csv', index=False)

model = LinearRegression()
r2_mh_dti = model.fit(all_g_mh_dti_all['g_pred_mh'].values.reshape(-1, 1), all_g_mh_dti_all['g_real']).score(all_g_mh_dti_all['g_pred_mh'].values.reshape(-1, 1), all_g_mh_dti_all['g_real'])
r2_dti = model.fit(all_g_mh_dti_all['g_pred_dti_all'].values.reshape(-1, 1), all_g_mh_dti_all['g_real']).score(all_g_mh_dti_all['g_pred_dti_all'].values.reshape(-1, 1), all_g_mh_dti_all['g_real'])
r2_mh_and_dti = model.fit(pd.concat([all_g_mh_dti_all['g_pred_dti_all'], all_g_mh_dti_all['g_pred_mh']], axis=1), all_g_mh_dti_all['g_real']).score(pd.concat([all_g_mh_dti_all['g_pred_dti_all'], all_g_mh_dti_all['g_pred_mh']], axis=1), all_g_mh_dti_all['g_real'])

#### Run CA

In [ ]:
all_g_mh_dti_all = pd.read_csv('/commonality_analysis/dti/g_real_pred_mh_dti-all.csv')
r2_mh_dti = commonality_analysis(all_g_mh_dti_all['g_pred_mh'], all_g_mh_dti_all['g_real'])
r2_dti = commonality_analysis(all_g_mh_dti_all['g_pred_dti_all'], all_g_mh_dti_all['g_real'])
r2_mh_and_dti = commonality_analysis(pd.concat([all_g_mh_dti_all['g_pred_dti_all'], all_g_mh_dti_all['g_pred_mh']], axis=1), all_g_mh_dti_all['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

u_mh_dti = r2_mh_and_dti - r2_dti
u_dti = r2_mh_and_dti - r2_mh_dti

c_mh_and_dti = r2_mh_and_dti - u_mh_dti - u_dti

print("R squared for MH:", r2_mh_dti.round(3)) # f"{r2_mh_m:.5f}")
print("R squared for DTI All", r2_dti.round(3))
print("R squared for MH and DTI All", r2_mh_and_dti.round(3))

print('_______________')
print("Unique variance for MH:", u_mh_dti.round(3)) #"{:.5f}".format(u_mh_dti)) # "{:.5f}".format(u_mh_for_dti)
print("Unique variance for DTI All:", u_dti.round(3)) #f"{u_rs:.5f}")

print('_______________')
print("Common variance for MH and DTI All:", c_mh_and_dti.round(3))
print('_______________')
perc_mh = u_mh_dti / r2_mh_and_dti * 100 #u_mh_dti / R squared for MH and DTI All
perc_dti = u_dti / r2_mh_and_dti * 100 #u_rs / R squared for MH and DTI All
perc_c = c_mh_and_dti / r2_mh_and_dti * 100 #c_mh_rs / R squared for MH and DTI All

print(f"Proportion of unique variance attributed to MH is {perc_mh.round(2)}")
print(f"Proportion of unique variance attributed to DTI All is {perc_dti.round(2)}")
print(f"Proportion of common variance attributed to MH and DTI All is {perc_c.round(2)}")

# DTI PLS

In [ ]:
# 1 Match g: original, from mental health, and from DTI
folds = [f'{i}' for i in range(5)]
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pls_g_mh_dti_concat = []
for fold in folds:
    g_pred_pls_test_dti = pd.read_csv(f'/PLS/brain/stacking/features_test_level1_stacked/dti_all/features_test_level1_g_matched_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    pls_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_pls_test_dti, on='eid').drop(columns=['eid'])
    #pls_g.columns = ['g_real', 'g_pred_mh', 'g_pred_dti_ts']
    pls_g_mh_dti_concat.append(pls_g)
    pls_g_mh_dti = pd.concat(pls_g_mh_dti_concat, axis=0, ignore_index=True)
    pls_g_mh_dti.to_csv('/commonality_analysis/dti/pls/g_real_pred_mh_dti-pls.csv', index=False)

#### Run CA

In [ ]:
# 2 Extract R2 with commonality_analysis function
pls_g_mh_dti = pd.read_csv('/commonality_analysis/dti/pls/g_real_pred_mh_dti-pls.csv')
r2_mh_from_dti_pls = LinearRegression().fit(pls_g_mh_dti['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_dti['g']).score(pls_g_mh_dti['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_dti['g'])

dti_cols= pls_g_mh_dti.columns[2:].to_list()

r2_dti = {}
for col in dti_cols:
    r2_score = commonality_analysis(pls_g_mh_dti[col], pls_g_mh_dti['g'])
    r2_dti[col] = r2_score
    globals()[f'r2_{col}'] = r2_score

# Iterate through each column to combine with 'g_pred_mh'
r2_mh_and_dti = {}
for col in dti_cols:
    combined_columns = pd.concat([pls_g_mh_dti[col], pls_g_mh_dti['g_pred_mh']], axis=1)
    r2_combined = commonality_analysis(combined_columns, pls_g_mh_dti['g'])
    r2_mh_and_dti[f'r2_mh_and_{col}'] = r2_combined
    # Create a variable for each combined R^2 score
    globals()[f'r2_mh_and_{col}'] = r2_combined

In [ ]:
# Save R2 dictionaries
with open('/commonality_analysis/dti/pls/r2_dti.pkl', 'wb') as f:
    pickle.dump(r2_dti, f)

with open('/commonality_analysis/dti/pls/r2_mh_and_dti.pkl', 'wb') as f:
    pickle.dump(r2_mh_and_dti, f)

In [ ]:
# 3 Calculate  unique and common variances
unique_mh = {}
unique_dti = {}
common_mh_dti = {}

for key in r2_mh_and_dti:
    dti_key = key.replace('r2_mh_and_', '')
    u_mh = r2_mh_and_dti[key] - r2_dti[dti_key]
    u_dti = r2_mh_and_dti[key] - r2_mh_from_dti_pls #[key]
    c_mh_and_dti = r2_mh_and_dti[key] - u_mh - u_dti
    
    unique_mh[f'u_mh_from_{dti_key}'] = u_mh
    unique_dti[f'u_dti_{dti_key}'] = u_dti
    common_mh_dti[f'c_mh_and_dti_{dti_key}'] = c_mh_and_dti

# Print results
for key in r2_mh_and_dti:
    dti_key = key.replace('r2_mh_and_', '')
    print(f"R squared for {dti_key}: {r2_dti[dti_key]:.3f}")
    print(f"R squared for MH and {dti_key}: {r2_mh_and_dti[key]:.3f}")
    print(f"Unique variance for MH ({dti_key}): {unique_mh[f'u_mh_from_{dti_key}']:.3f}")
    print(f"Unique variance for dti ({dti_key}): {unique_dti[f'u_dti_{dti_key}']:.3f}")
    print(f"Common variance for MH and dti ({dti_key}): {common_mh_dti[f'c_mh_and_dti_{dti_key}']:.3f}")
    print('_______________')

# Calculate proportions
for key in r2_mh_and_dti:
    dti_key = key.replace('r2_mh_and_', '')
    perc_mh = unique_mh[f'u_mh_from_{dti_key}'] / r2_mh_and_dti[key] * 100
    perc_dti = unique_dti[f'u_dti_{dti_key}'] / r2_mh_and_dti[key] * 100
    perc_c = common_mh_dti[f'c_mh_and_dti_{dti_key}'] / r2_mh_and_dti[key] * 100
    
    print(f"Proportion of unique variance attributed to MH ({dti_key}): {perc_mh:.2f}%")
    print(f"Proportion of unique variance attributed to dti ({dti_key}): {perc_dti:.2f}%")
    print(f"Proportion of common variance attributed to MH and dti ({dti_key}): {perc_c:.2f}%")
    print('_______________')

In [ ]:
# Save unique and common variances
with open('/commonality_analysis/dti/pls/unique_mh.pkl', 'wb') as f:
    pickle.dump(unique_mh, f)

with open('/commonality_analysis/dti/pls/unique_dti.pkl', 'wb') as f:
    pickle.dump(unique_dti, f)

with open('/commonality_analysis/dti/pls/common_mh_dti.pkl', 'wb') as f:
    pickle.dump(common_mh_dti, f)

#### Upload CA variables for DTI

In [ ]:
# R2
pls_g_mh_dti = pd.read_csv('/commonality_analysis/dti/pls/g_real_pred_mh_dti-pls.csv')
r2_mh_from_dti_pls = LinearRegression().fit(pls_g_mh_dti['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_dti['g']).score(pls_g_mh_dti['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_dti['g'])

# Load R2 dictionaries from pickle files
with open('/commonality_analysis/dti/pls/r2_dti.pkl', 'rb') as f:
    r2_dti = pickle.load(f)

with open('/commonality_analysis/dti/pls/r2_mh_and_dti.pkl', 'rb') as f:
    r2_mh_and_dti = pickle.load(f)

# Extract variables
for col in r2_dti:
    globals()[col] = round(r2_dti[col] * 100, 3)
    r2_dti[col] = round(r2_dti[col] * 100, 3)  # Update the dictionary with rounded values

for col in r2_mh_and_dti:
    globals()[f'r2_mh_and_{col}'] = round(r2_mh_and_dti[col] * 100, 3)
    r2_mh_and_dti[col] = round(r2_mh_and_dti[col] * 100, 3)

print(r2_dti)
print(r2_mh_and_dti)

In [ ]:
# U/C
with open('/commonality_analysis/dti/pls/unique_mh.pkl', 'rb') as f:
    unique_mh = pickle.load(f)

with open('/commonality_analysis/dti/pls/unique_dti.pkl', 'rb') as f:
    unique_dti = pickle.load(f)

with open('/commonality_analysis/dti/pls/common_mh_dti.pkl', 'rb') as f:
    common_mh_dti = pickle.load(f)

# Extract variables
for key in unique_mh:
    globals()[key] = round(unique_mh[key] * 100, 3)
    unique_mh[key] = round(unique_mh[key] * 100, 3)

for key in unique_dti:
    globals()[key] = round(unique_dti[key] * 100, 3)
    unique_dti[key] = round(unique_dti[key] * 100, 3) 

for key in common_mh_dti:
    globals()[key] = round(common_mh_dti[key] * 100, 3)
    common_mh_dti[key] = round(common_mh_dti[key] * 100, 3)

print(unique_mh)
print(unique_dti)
print(common_mh_dti)

##########################################################

# T1/T2 Stacked

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

all_g_mh_t1_concat = []

for fold in folds:
    g_pred_t1_svr = pd.read_csv(f'/PLS/brain/stacking/g/T1_T2_whole_brain_target_pred_2nd_level_svr_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_t1_svr, on='eid').drop(columns=['eid'])
    all_g.columns = ['g_real', 'g_pred_mh', 'g_pred_t1']
    all_g_mh_t1_concat.append(all_g)
    all_g_mh_t1 = pd.concat(all_g_mh_t1_concat, axis=0, ignore_index=True)
    all_g_mh_t1.to_csv('/commonality_analysis/struct/g_real_pred_mh_t1.csv', index=False)

model = LinearRegression()
r2_mh_t1 = model.fit(all_g_mh_t1['g_pred_mh'].values.reshape(-1, 1), all_g_mh_t1['g_real'].values.reshape(-1, 1)).score(all_g_mh_t1['g_pred_mh'].values.reshape(-1, 1), all_g_mh_t1['g_real'].values.reshape(-1, 1))
r2_t1 = model.fit(all_g_mh_t1['g_pred_t1'].values.reshape(-1, 1), all_g_mh_t1['g_real'].values.reshape(-1, 1)).score(all_g_mh_t1['g_pred_t1'].values.reshape(-1, 1), all_g_mh_t1['g_real'].values.reshape(-1, 1))
r2_mh_and_t1 = model.fit(pd.concat([all_g_mh_t1['g_pred_t1'], all_g_mh_t1['g_pred_mh']], axis=1), all_g_mh_t1['g_real']).score(pd.concat([all_g_mh_t1['g_pred_t1'], all_g_mh_t1['g_pred_mh']], axis=1), all_g_mh_t1['g_real'].values.reshape(-1, 1))

#### Run CA

In [ ]:
all_g_mh_t1 = pd.read_csv('/commonality_analysis/struct/g_real_pred_mh_t1.csv')
r2_mh_t1 = commonality_analysis(all_g_mh_t1['g_pred_mh'], all_g_mh_t1['g_real'])
r2_t1 = commonality_analysis(all_g_mh_t1['g_pred_t1'], all_g_mh_t1['g_real'])
r2_mh_and_t1 = commonality_analysis(pd.concat([all_g_mh_t1['g_pred_t1'], all_g_mh_t1['g_pred_mh']], axis=1), all_g_mh_t1['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

u_mh_t1 = r2_mh_and_t1 - r2_t1
u_t1 = r2_mh_and_t1 - r2_mh_t1

c_mh_and_t1 = r2_mh_and_t1 - u_mh_t1 - u_t1

print("R squared for MH:", r2_mh_t1.round(3)) # f"{r2_mh_m:.5f}")
print("R squared for T1", r2_t1.round(3))
print("R squared for MH and T1", r2_mh_and_t1.round(3))

print('_______________')
print("Unique variance for MH:", u_mh_t1.round(3))
print("Unique variance for T1:", u_t1.round(3))

print('_______________')
print("Common variance for MH and T1:", c_mh_and_t1.round(3))
print('_______________')
perc_mh = u_mh_t1 / r2_mh_and_t1 * 100
perc_t1 = u_t1 / r2_mh_and_t1 * 100
perc_c = c_mh_and_t1 / r2_mh_and_t1 * 100

print(f"Proportion of unique variance attributed to MH is {perc_mh.round(2)}")
print(f"Proportion of unique variance attributed to T1 is {perc_t1.round(2)}")
print(f"Proportion of common variance attributed to MH and T1 is {perc_c.round(2)}")

# T1/T2 PLS

In [ ]:
# 1 Match g: original, from mental health, and from sMRI
folds = [f'{i}' for i in range(5)]
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pls_g_mh_t1_t2_concat = []
for fold in folds:
    g_pred_pls_test_t1_t2 = pd.read_csv(f'/PLS/brain/stacking/features_test_level1_stacked/t1_t2_struct/features_test_level1_g_matched_fold_{fold}.csv')
    g_pred_mh = g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    pls_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_pls_test_t1_t2, on='eid').drop(columns=['eid'])
    pls_g_mh_t1_t2_concat.append(pls_g)
    pls_g_mh_t1_t2 = pd.concat(pls_g_mh_t1_t2_concat, axis=0, ignore_index=True)
    pls_g_mh_t1_t2.to_csv('/commonality_analysis/struct/pls/g_real_pred_mh_t1_t2-pls.csv', index=False)

In [ ]:
# 2 Extract R2 with commonality_analysis function
pls_g_mh_t1_t2 = pd.read_csv('/commonality_analysis/struct/pls/g_real_pred_mh_t1_t2-pls.csv')
r2_mh_from_t1_t2_pls = LinearRegression().fit(pls_g_mh_t1_t2['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_t1_t2['g']).score(pls_g_mh_t1_t2['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_t1_t2['g'])

t1_t2_cols= pls_g_mh_t1_t2.columns[2:].to_list()

r2_t1_t2 = {}
for col in t1_t2_cols:
    r2_score = commonality_analysis(pls_g_mh_t1_t2[col], pls_g_mh_t1_t2['g'])
    r2_t1_t2[col] = r2_score
    globals()[f'r2_{col}'] = r2_score

# Iterate through each column to combine with 'g_pred_mh'
r2_mh_and_t1_t2 = {}
for col in t1_t2_cols:
    combined_columns = pd.concat([pls_g_mh_t1_t2[col], pls_g_mh_t1_t2['g_pred_mh']], axis=1)
    r2_combined = commonality_analysis(combined_columns, pls_g_mh_t1_t2['g'])
    r2_mh_and_t1_t2[f'r2_mh_and_{col}'] = r2_combined
    # Create a variable for each combined R^2 score
    globals()[f'r2_mh_and_{col}'] = r2_combined

In [ ]:
# Save R2 dictionaries
with open('/commonality_analysis/struct/pls/r2_t1_t2.pkl', 'wb') as f:
    pickle.dump(r2_t1_t2, f)

with open('/commonality_analysis/struct/pls/r2_mh_and_t1_t2.pkl', 'wb') as f:
    pickle.dump(r2_mh_and_t1_t2, f)

In [ ]:
# 3 Calculate  unique and common variances
unique_mh = {}
unique_t1_t2 = {}
common_mh_t1_t2 = {}

for key in r2_mh_and_t1_t2:
    t1_t2_key = key.replace('r2_mh_and_', '')
    u_mh = r2_mh_and_t1_t2[key] - r2_t1_t2[t1_t2_key]
    u_t1_t2 = r2_mh_and_t1_t2[key] - r2_mh_from_t1_t2_pls #[key]
    c_mh_and_t1_t2 = r2_mh_and_t1_t2[key] - u_mh - u_t1_t2
    
    unique_mh[f'u_mh_from_{t1_t2_key}'] = u_mh
    unique_t1_t2[f'u_t1_t2_{t1_t2_key}'] = u_t1_t2
    common_mh_t1_t2[f'c_mh_and_t1_t2_{t1_t2_key}'] = c_mh_and_t1_t2

# Print results
for key in r2_mh_and_t1_t2:
    t1_t2_key = key.replace('r2_mh_and_', '')
    print(f"R squared for {t1_t2_key}: {r2_t1_t2[t1_t2_key]:.3f}")
    print(f"R squared for MH and {t1_t2_key}: {r2_mh_and_t1_t2[key]:.3f}")
    print(f"Unique variance for MH ({t1_t2_key}): {unique_mh[f'u_mh_from_{t1_t2_key}']:.3f}")
    print(f"Unique variance for t1_t2 ({t1_t2_key}): {unique_t1_t2[f'u_t1_t2_{t1_t2_key}']:.3f}")
    print(f"Common variance for MH and t1_t2 ({t1_t2_key}): {common_mh_t1_t2[f'c_mh_and_t1_t2_{t1_t2_key}']:.3f}")
    print('_______________')

# Calculate proportions
for key in r2_mh_and_t1_t2:
    t1_t2_key = key.replace('r2_mh_and_', '')
    perc_mh = unique_mh[f'u_mh_from_{t1_t2_key}'] / r2_mh_and_t1_t2[key] * 100
    perc_t1_t2 = unique_t1_t2[f'u_t1_t2_{t1_t2_key}'] / r2_mh_and_t1_t2[key] * 100
    perc_c = common_mh_t1_t2[f'c_mh_and_t1_t2_{t1_t2_key}'] / r2_mh_and_t1_t2[key] * 100
    
    print(f"Proportion of unique variance attributed to MH ({t1_t2_key}): {perc_mh:.2f}%")
    print(f"Proportion of unique variance attributed to t1_t2 ({t1_t2_key}): {perc_t1_t2:.2f}%")
    print(f"Proportion of common variance attributed to MH and t1_t2 ({t1_t2_key}): {perc_c:.2f}%")
    print('_______________')

In [ ]:
# Save unique and common variances
with open('/commonality_analysis/struct/pls/unique_mh.pkl', 'wb') as f:
    pickle.dump(unique_mh, f)

with open('/commonality_analysis/struct/pls/unique_t1_t2.pkl', 'wb') as f:
    pickle.dump(unique_t1_t2, f)

with open('/commonality_analysis/struct/pls/common_mh_t1_t2.pkl', 'wb') as f:
    pickle.dump(common_mh_t1_t2, f)

#### Upload CA variables for T1/T2

In [ ]:
# R2
pls_g_mh_t1_t2 = pd.read_csv('/commonality_analysis/struct/pls/g_real_pred_mh_t1_t2-pls.csv')
r2_mh_from_t1_t2_pls = LinearRegression().fit(pls_g_mh_t1_t2['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_t1_t2['g']).score(pls_g_mh_t1_t2['g_pred_mh'].values.reshape(-1, 1), pls_g_mh_t1_t2['g'])

# Load R2 dictionaries from pickle files
with open('/commonality_analysis/struct/pls/r2_t1_t2.pkl', 'rb') as f:
    r2_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/r2_mh_and_t1_t2.pkl', 'rb') as f:
    r2_mh_and_t1_t2 = pickle.load(f)

# Extract variables
for col in r2_t1_t2:
    globals()[col] = round(r2_t1_t2[col] * 100, 3)
    r2_t1_t2[col] = round(r2_t1_t2[col] * 100, 3)  # Update the dictionary with rounded values

for col in r2_mh_and_t1_t2:
    globals()[f'r2_mh_and_{col}'] = round(r2_mh_and_t1_t2[col] * 100, 3)
    r2_mh_and_t1_t2[col] = round(r2_mh_and_t1_t2[col] * 100, 3)

print(r2_t1_t2)
print(r2_mh_and_t1_t2)

In [ ]:
# U/C
with open('/commonality_analysis/struct/pls/unique_mh.pkl', 'rb') as f:
    unique_mh = pickle.load(f)

with open('/commonality_analysis/struct/pls/unique_t1_t2.pkl', 'rb') as f:
    unique_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/common_mh_t1_t2.pkl', 'rb') as f:
    common_mh_t1_t2 = pickle.load(f)

# Extract variables
for key in unique_mh:
    globals()[key] = round(unique_mh[key] * 100, 3)
    unique_mh[key] = round(unique_mh[key] * 100, 3)  # Update the dictionary with rounded values

for key in unique_t1_t2:
    globals()[key] = round(unique_t1_t2[key] * 100, 3)
    unique_t1_t2[key] = round(unique_t1_t2[key] * 100, 3) 

for key in common_mh_t1_t2:
    globals()[key] = round(common_mh_t1_t2[key] * 100, 3)
    common_mh_t1_t2[key] = round(common_mh_t1_t2[key] * 100, 3)

print(unique_mh)
print(unique_t1_t2)
print(common_mh_t1_t2)

# All Modalities Stacked

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = ['0','1','2','3','4']

all_g_mh_mri_all_concat = []

for fold in folds:
    g_pred_mri_all = pd.read_csv(f'/PLS/brain/stacking/g/All_modalities_target_pred_2nd_level_xgb_test_fold_{fold}.csv')
    g_pred_mh = g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_mri_all, on='eid').drop(columns=['eid'])
    all_g.columns = ['g_real', 'g_pred_mh', 'g_pred_mri_all']
    all_g_mh_mri_all_concat.append(all_g)
    all_g_mh_mri_all = pd.concat(all_g_mh_mri_all_concat, axis=0, ignore_index=True)
    all_g_mh_mri_all.to_csv('/commonality_analysis/all/g_real_pred_mh_mri-all.csv', index=False)

model = LinearRegression()
r2_mh_mri = model.fit(all_g_mh_mri_all['g_pred_mh'].values.reshape(-1, 1), all_g_mh_mri_all['g_real'].values.reshape(-1, 1)).score(all_g_mh_mri_all['g_pred_mh'].values.reshape(-1, 1), all_g_mh_mri_all['g_real'].values.reshape(-1, 1))
r2_mri = model.fit(all_g_mh_mri_all['g_pred_mri_all'].values.reshape(-1, 1), all_g_mh_mri_all['g_real'].values.reshape(-1, 1)).score(all_g_mh_mri_all['g_pred_mri_all'].values.reshape(-1, 1), all_g_mh_mri_all['g_real'].values.reshape(-1, 1))
r2_mh_and_mri = model.fit(pd.concat([all_g_mh_mri_all['g_pred_mri_all'], all_g_mh_mri_all['g_pred_mh']], axis=1), all_g_mh_mri_all['g_real']).score(pd.concat([all_g_mh_mri_all['g_pred_mri_all'], all_g_mh_mri_all['g_pred_mh']], axis=1), all_g_mh_mri_all['g_real'].values.reshape(-1, 1))

#### Run CA

In [ ]:
all_g_mh_mri_all = pd.read_csv('/commonality_analysis/all/g_real_pred_mh_mri-all.csv')
r2_mh_mri = commonality_analysis(all_g_mh_mri_all['g_pred_mh'], all_g_mh_mri_all['g_real'])
r2_mri = commonality_analysis(all_g_mh_mri_all['g_pred_mri_all'], all_g_mh_mri_all['g_real'])
r2_mh_and_mri = commonality_analysis(pd.concat([all_g_mh_mri_all['g_pred_mri_all'], all_g_mh_mri_all['g_pred_mh']], axis=1), all_g_mh_mri_all['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

u_mh_mri = r2_mh_and_mri - r2_mri
u_mri = r2_mh_and_mri - r2_mh_mri

c_mh_and_mri = r2_mh_and_mri - u_mh_mri - u_mri

print("R squared for MH:", r2_mh_mri.round(3)) # f"{r2_mh_mri_m:.5f}")
print("R squared for MRI", r2_mri.round(3))
print("R squared for MH and MRI", r2_mh_and_mri.round(3))

print('_______________')
print("Unique variance for MH:", u_mh_mri.round(3)) #"{:.5f}".format(u_mh_mri)) # "{:.5f}".format(u_mh_mri)
print("Unique variance for MRI:", u_mri.round(3)) #f"{u_rs:.5f}")

print('_______________')
print("Common variance for MH and MRI:", c_mh_and_mri.round(3))
print('_______________')
perc_mh = u_mh_mri / r2_mh_and_mri * 100 #u_mh_mri / R squared for MH and T1 plus T2
perc_mri = u_mri / r2_mh_and_mri * 100 #u_rs / R squared for MH and T1 plus T2
perc_c = c_mh_and_mri / r2_mh_and_mri * 100 #c_mh_rs / R squared for MH and T1 plus T2

print(f"Proportion of unique variance attributed to MH is {perc_mh.round(2)}")
print(f"Proportion of unique variance attributed to MRI is {perc_mri.round(2)}")
print(f"Proportion of common variance attributed to MH and MRI is {perc_c.round(2)}")

##########################################################

# Combine all PLS CA results in one dataframe

In [ ]:
# Upload variables
# DTI
with open('/commonality_analysis/dti/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_dti = pickle.load(f)

with open('/commonality_analysis/dti/pls/unique_dti.pkl', 'rb') as f:
    unique_dti = pickle.load(f)

with open('/commonality_analysis/dti/pls/common_mh_dti.pkl', 'rb') as f:
    common_mh_dti = pickle.load(f)

# RS
with open('/commonality_analysis/rs/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_rs = pickle.load(f)

with open('/commonality_analysis/rs/pls/unique_rs.pkl', 'rb') as f:
    unique_rs = pickle.load(f)

with open('/commonality_analysis/rs/pls/common_mh_rs.pkl', 'rb') as f:
    common_mh_rs = pickle.load(f)

# T1/T2
with open('/commonality_analysis/struct/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/unique_t1_t2.pkl', 'rb') as f:
    unique_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/common_mh_t1_t2.pkl', 'rb') as f:
    common_mh_t1_t2 = pickle.load(f)

In [ ]:
# Combine all into one data frame: long format
commonality_pls = {}

# DTI
for key in unique_mh_dti:
    commonality_pls[f'{key}'] = round(unique_mh_dti[key] * 100, 3)
for key in unique_dti:
    commonality_pls[f'{key}'] = round(unique_dti[key] * 100, 3)
for key in common_mh_dti:
    commonality_pls[f'{key}'] = round(common_mh_dti[key] * 100, 3)

# RS
for key in unique_mh_rs:
    commonality_pls[f'{key}'] = round(unique_mh_rs[key] * 100, 3)
for key in unique_rs:
    commonality_pls[f'{key}'] = round(unique_rs[key] * 100, 3)
for key in common_mh_rs:
    commonality_pls[f'{key}'] = round(common_mh_rs[key] * 100, 3)

# T1/T2
for key in unique_mh_t1_t2:
    commonality_pls[f'{key}'] = round(unique_mh_t1_t2[key] * 100, 3)
for key in unique_t1_t2:
    commonality_pls[f'{key}'] = round(unique_t1_t2[key] * 100, 3)
for key in common_mh_t1_t2:
    commonality_pls[f'{key}'] = round(common_mh_t1_t2[key] * 100, 3)


# Convert the dictionary to a DataFrame
commonality_pls_df = pd.DataFrame.from_dict(commonality_pls, orient='index', columns=['Value'])
commonality_pls_df


In [ ]:
# Combine all into one data frame: wide format
commonality_pls_dict = {}

# DTI
commonality_metrics_dti = {}
for key in unique_mh_dti:
    modality = key.replace('u_mh_from_', '')
    commonality_pls_dict[modality] = {
        'Unique: Mental Health': round(unique_mh_dti[key] * 100, 3),
        'Unique: MRI': round(unique_dti[key.replace('u_mh_from_', 'u_dti_')] * 100, 3),
        'Common': round(common_mh_dti[key.replace('u_mh_from_', 'c_mh_and_dti_')] * 100, 3)
    }
    commonality_metrics_dti[modality] = {
        'Unique: Mental Health': round(unique_mh_dti[key] * 100, 3),
        'Unique: MRI': round(unique_dti[key.replace('u_mh_from_', 'u_dti_')] * 100, 3),
        'Common': round(common_mh_dti[key.replace('u_mh_from_', 'c_mh_and_dti_')] * 100, 3)
    }

# T1/T2
commonality_metrics_struct = {}
for key in unique_mh_t1_t2:
    modality = key.replace('u_mh_from_', '')
    commonality_pls_dict[modality] = {
        'Unique: Mental Health': round(unique_mh_t1_t2[key] * 100, 3),
        'Unique: MRI': round(unique_t1_t2[key.replace('u_mh_from_', 'u_t1_t2_')] * 100, 3),
        'Common': round(common_mh_t1_t2[key.replace('u_mh_from_', 'c_mh_and_t1_t2_')] * 100, 3)
    }
    commonality_metrics_struct[modality] = {
        'Unique: Mental Health': round(unique_mh_t1_t2[key] * 100, 3),
        'Unique: MRI': round(unique_t1_t2[key.replace('u_mh_from_', 'u_t1_t2_')] * 100, 3),
        'Common': round(common_mh_t1_t2[key.replace('u_mh_from_', 'c_mh_and_t1_t2_')] * 100, 3)
    }

# RS
commonality_metrics_rs = {}
for key in unique_mh_rs:
    modality = key.replace('u_mh_from_', '')
    commonality_pls_dict[modality] = {
        'Unique: Mental Health': round(unique_mh_rs[key] * 100, 3),
        'Unique: MRI': round(unique_rs[key.replace('u_mh_from_', 'u_rs_')] * 100, 3),
        'Common': round(common_mh_rs[key.replace('u_mh_from_', 'c_mh_and_rs_')] * 100, 3)
    }
    commonality_metrics_rs[modality] = {
        'Unique: Mental Health': round(unique_mh_rs[key] * 100, 3),
        'Unique: MRI': round(unique_rs[key.replace('u_mh_from_', 'u_rs_')] * 100, 3),
        'Common': round(common_mh_rs[key.replace('u_mh_from_', 'c_mh_and_rs_')] * 100, 3)
    }

# Convert the dictionary to a DataFrame
commonality_metrics_df = pd.DataFrame.from_dict(commonality_pls_dict, orient='index')
commonality_metrics_dti_df = pd.DataFrame.from_dict(commonality_metrics_dti, orient='index')
commonality_metrics_dti_df.to_csv('/commonality_analysis/commonality_metrics_dti_df.csv')

commonality_metrics_struct_df = pd.DataFrame.from_dict(commonality_metrics_struct, orient='index')
commonality_metrics_struct_df.to_csv('/commonality_analysis/commonality_metrics_struct_df.csv')

commonality_metrics_rs_df = pd.DataFrame.from_dict(commonality_metrics_rs, orient='index')
commonality_metrics_rs_df.to_csv('/commonality_analysis/commonality_metrics_rs_df.csv')


with open('/commonality_analysis/commonality_metrics_dti.pkl', 'wb') as f: #to open, change wb to rb, and dump to load
    pickle.dump(commonality_metrics_dti, f)

with open('/commonality_analysis/commonality_metrics_struct_df.pkl', 'wb') as f: #to open, change wb to rb, and dump to load
    pickle.dump(commonality_metrics_struct_df, f)

with open('/commonality_analysis/commonality_metrics_rs_df.pkl', 'wb') as f: #to open, change wb to rb, and dump to load
    pickle.dump(commonality_metrics_rs_df, f)

    
commonality_metrics_df.to_csv('/commonality_analysis/commonality_metrics_df.csv')
commonality_metrics_df

# Combine all Stack CA results in one dataframe

Extract commonality metrics and save

In [ ]:
u_mh_rs_na = u_mh_rs.astype(float).round(4)*100
u_rs_na = u_rs.astype(float).round(4)*100
c_mh_and_rs_na = c_mh_and_rs.astype(float).round(4)*100
u_mh_dti_na = u_mh_dti.astype(float).round(4)*100
u_dti_na = u_dti.astype(float).round(4)*100
c_mh_and_dti_na = c_mh_and_dti.astype(float).round(4)*100
u_mh_t1_na = u_mh_t1.astype(float).round(4)*100
u_t1_na = u_t1.astype(float).round(4)*100
c_mh_and_t1_na = (c_mh_and_t1.astype(float).round(4)*100).round(1)
u_mh_mri_na = u_mh_mri.astype(float).round(4)*100
u_mri_na = u_mri.astype(float).round(4)*100
c_mh_and_mri_na = c_mh_and_mri.astype(float).round(4)*100

In [ ]:
commonality_metrics_no_demo_dict = {
'u_mh_rs_na': u_mh_rs_na,
'u_rs_na': u_rs_na,
'c_mh_and_rs_na': c_mh_and_rs_na,
'u_mh_dti_na': u_mh_dti_na,
'u_dti_na': u_dti_na,
'c_mh_and_dti_na': c_mh_and_dti_na,
'u_mh_t1_na': u_mh_t1_na,
'u_t1_na': u_t1_na,
'c_mh_and_t1_na': c_mh_and_t1_na,
'u_mh_mri_na': u_mh_mri_na,
'u_mri_na': u_mri_na,
'c_mh_and_mri_na': c_mh_and_mri_na
}

commonality_metrics_no_demo = pd.DataFrame(commonality_metrics_no_demo_dict, index=[0])
commonality_metrics_no_demo.to_csv('/commonality_analysis/commonality_metrics_no_demo.csv', index=False)
commonality_metrics_no_demo

In [ ]:
# Extract the values from the DataFrame
u_mh_rs_na = commonality_metrics_no_demo['u_mh_rs_na'].iloc[0]
u_rs_na = commonality_metrics_no_demo['u_rs_na'].iloc[0]
c_mh_and_rs_na = commonality_metrics_no_demo['c_mh_and_rs_na'].iloc[0]
u_mh_dti_na = commonality_metrics_no_demo['u_mh_dti_na'].iloc[0]
u_dti_na = commonality_metrics_no_demo['u_dti_na'].iloc[0]
c_mh_and_dti_na = commonality_metrics_no_demo['c_mh_and_dti_na'].iloc[0]
u_mh_t1_na = commonality_metrics_no_demo['u_mh_t1_na'].iloc[0]
u_t1_na = commonality_metrics_no_demo['u_t1_na'].iloc[0]
c_mh_and_t1_na = commonality_metrics_no_demo['c_mh_and_t1_na'].iloc[0]
u_mh_mri_na = commonality_metrics_no_demo['u_mh_mri_na'].iloc[0]
u_mri_na = commonality_metrics_no_demo['u_mri_na'].iloc[0]
c_mh_and_mri_na = commonality_metrics_no_demo['c_mh_and_mri_na'].iloc[0]

# Print the extracted values
print(f"u_mh_rs_na: {u_mh_rs_na.round(3)}")
print(f"u_rs_na: {u_rs_na.round(3)}")
print(f"c_mh_and_rs_na: {c_mh_and_rs_na.round(3)}")
print(f"u_mh_dti_na: {u_mh_dti_na.round(3)}")
print(f"u_dti_na: {u_dti_na.round(3)}")
print(f"c_mh_and_dti_na: {c_mh_and_dti_na.round(3)}")
print(f"u_mh_t1_na: {u_mh_t1_na.round(3)}")
print(f"u_t1_na: {u_t1_na.astype(float).round(3)}")
print(f"c_mh_and_t1_na: {c_mh_and_t1_na.round(3)}")
print(f"u_mh_mri_na: {u_mh_mri_na.round(3)}")
print(f"u_mri_na: {u_mri_na.round(3)}")
print(f"c_mh_and_mri_na: {c_mh_and_mri_na.round(3)}")

In [ ]:
data = {
    'dwMRI': {
        'Unique: Mental Health': (commonality_metrics_no_demo['u_mh_dti_na'].iloc[0]),
        'Unique: MRI': (commonality_metrics_no_demo['u_dti_na'].iloc[0]),
        'Common': (commonality_metrics_no_demo['c_mh_and_dti_na'].iloc[0])
    },
    'rsMRI': {
        'Unique: Mental Health': (commonality_metrics_no_demo['u_mh_rs_na'].iloc[0]),
        'Unique: MRI': (commonality_metrics_no_demo['u_rs_na'].iloc[0]),
        'Common': (commonality_metrics_no_demo['c_mh_and_rs_na'].iloc[0])
    },
    'T1w/T2w MRI': {
        'Unique: Mental Health': (commonality_metrics_no_demo['u_mh_t1_na'].iloc[0]),
        'Unique: MRI': (commonality_metrics_no_demo['u_t1_na'].iloc[0]),
        'Common': (commonality_metrics_no_demo['c_mh_and_t1_na'].iloc[0])
    },
    'Stacked MRI': {
        'Unique: Mental Health': (commonality_metrics_no_demo['u_mh_mri_na'].iloc[0]),
        'Unique: MRI': (commonality_metrics_no_demo['u_mri_na'].iloc[0]),
        'Common': (commonality_metrics_no_demo['c_mh_and_mri_na'].iloc[0])
    }
}

# Create the DataFrame
df = pd.DataFrame(data).T
df.to_csv('/commonality_analysis/commonality_metrics_no_demo_supplem_table.csv')
df

##  Calculate % of variance explained by MRI

In [ ]:
commonality_no_demo_df = pd.read_csv('/commonality_analysis/commonality_metrics_no_demo_supplem_table.csv').rename(columns = {'Unnamed: 0': 'Modality'})
commonality_no_demo_df['Var Exp MRI'] = commonality_no_demo_df['Common'].round(2) / (commonality_no_demo_df['Common'].round(2) + commonality_no_demo_df['Unique: Mental Health'].round(2))
commonality_no_demo_df['Var Exp MRI'] = (commonality_no_demo_df['Var Exp MRI'] * 100).round(2)
commonality_no_demo_df['Total Var'] = commonality_no_demo_df.iloc[:, 1:-1].sum(axis=1)
commonality_no_demo_df = commonality_no_demo_df.sort_values(by='Total Var', ascending = False)
commonality_no_demo_df.to_csv('/commonality_analysis/var_explained_dti_full.csv')
commonality_no_demo_df

# Stacked barplots for PLSR CA

## DTI

In [ ]:
# Rename phenotypes
base_rename_dict_dti = {'fa_tbss':'FA TBSS',
'fa_prob':'FA Prob.',
'md_tbss':'MD TBSS',
'md_prob':'MD Prob.',
'l1_tbss':'L1 TBSS',
'l1_prob':'L1 Prob.',
'l2_tbss':'L2 TBSS',
'l2_prob':'L2 Prob.',
'l3_tbss':'L3 TBSS',
'l3_prob':'L3 Prob.',
'mo_tbss':'MO TBSS',
'mo_prob':'MO Prob.',
'od_tbss':'OD TBSS',
'od_prob':'OD Prob.',
'icvf_tbss':'ICVF TBSS',
'icvf_prob':'ICVF Prob.',
'isovf_tbss':'ISOVF TBSS',
'isovf_prob':'ISOVF Prob.',

'31020_connectome_fa':'aparc.a2009s-I FA',
'31020_connectome_mean_length':'aparc.a2009s-I Mean Length',
'31020_connectome_sift2':'aparc.a2009s-I SIFT2',
'31020_connectome_streamline_count':'aparc.a2009s-I Streamline Count',

'31021_connectome_fa':'aparc-I FA',
'31021_connectome_mean_length':'aparc-I Mean Length',
'31021_connectome_sift2':'aparc-I SIFT2',
'31021_connectome_streamline_count':'aparc-I Streamline Count',

'31022_connectome_fa':'Glasser-I FA',
'31022_connectome_mean_length':'Glasser-I Mean Length',
'31022_connectome_sift2':'Glasser-I SIFT2',
'31022_connectome_streamline_count':'Glasser-I Streamline Count',

'31023_connectome_fa':'Glasser-IV FA',
'31023_connectome_mean_length':'Glasser-IV Mean Length',
'31023_connectome_sift2':'Glasser-IV SIFT2',
'31023_connectome_streamline_count':'Glasser-IV Streamline Count',

'31024_connectome_fa':'Schaefer200-I FA',
'31024_connectome_mean_length':'Schaefer200-I Mean Length',
'31024_connectome_sift2':'Schaefer200-I SIFT2',
'31024_connectome_streamline_count':'Schaefer200-I Streamline Count',

'31025_connectome_fa':'Schaefer500-IV FA',
'31025_connectome_mean_length':'Schaefer500-IV Mean Length',
'31025_connectome_sift2':'Schaefer500-IV SIFT2',
'31025_connectome_streamline_count_10M':'Schaefer500-IV Streamline Count'}

In [ ]:
# Load variables
with open('/commonality_analysis/dti/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_dti = pickle.load(f)

with open('/commonality_analysis/dti/pls/unique_dti.pkl', 'rb') as f:
    unique_dti = pickle.load(f)

with open('/commonality_analysis/dti/pls/common_mh_dti.pkl', 'rb') as f:
    common_mh_dti = pickle.load(f)
# Set variables
for key in unique_mh_dti: # unique for MH
    globals()[key] = round(unique_mh_dti[key] * 100, 3)
for key in unique_dti: # unique for rs modalities
    globals()[key] = round(unique_dti[key] * 100, 3)
for key in common_mh_dti: # common for MH and rs modalities
    globals()[key] = round(common_mh_dti[key] * 100, 3)

# Define x
modalities = list(unique_mh_dti.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in common_mh_dti.keys()])
y2 = np.array([globals()[f'{key}'] for key in unique_mh_dti.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_dti.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1] 

x_sorted = [base_rename_dict_dti.get(item, item) for item in np.array(x)[sorted_indices]] 
y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

plt.figure(figsize=(5, 10))
plt.barh(x_sorted, y1_sorted, linewidth=1, color= '#f56f5c', alpha=0.6) 
plt.barh(x_sorted, y2_sorted, left=y1_sorted, color='#B24745FF', linewidth=1, alpha=0.3)
plt.barh(x_sorted, y3_sorted, left=y1_sorted+y2_sorted, color= '#00A1D599', linewidth=1, alpha=0.3)


for i, (value_c, value_u_mh, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    plt.text(value_c / 2.2, i, f'{value_c:.1f}%', ha='left', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh / 2, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh + value_u_mri / 2, i, f'{value_u_mri:.1f}%', ha='center', va='center', color='black', fontsize=10)


plt.xlabel("% Variance Explained", fontsize=25, labelpad=20)
#plt.legend(["Common Variance: Mental Health + dwMRI", "Unique variance: Mental Health", "Unique variance: dwMRI"], fontsize = 10, ncol=3, loc='lower center', bbox_to_anchor=(0.5, -0.2))
plt.yticks(fontsize=12)
plt.xticks(fontsize=15)
plt.tick_params(axis='y', length=0)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(1.5))



plt.savefig("/CA-dwMRI.png",
            bbox_inches="tight", 
            pad_inches=1, 
            transparent=False, 
            facecolor="w", 
            edgecolor='w', 
            orientation='landscape',
            format='png')


plt.show()

## RS

In [ ]:
# Rename phenotypes
base_rename_dict_rs = {"aparc_s1_full_correlation":'aparc-I Func. Connectivity',
"aparc_2009_s1_full_correlation":'aparc.a2009s-I Func. Connectivity',
"glasser_s1_full_correlation":'Glasser-I Func. Connectivity',
"glasser_s4_full_correlation":'Glasser-IV Func. Connectivity',
"Schaefer7n200p_s1_full_correlation":'Schaefer200-I Func. Connectivity',
"Schaefer7n500p_s4_full_correlation":'Schaefer500-IV Func. Connectivity',
"amplitudes_21":'21 IC Amplitudes',
"amplitudes_55":'55 IC Amplitudes',
"tangent_matrices_21":'21 IC Func. Connectivity',
"tangent_matrices_55":'55 IC Func. Connectivity'}

In [ ]:
# Barplot
# Load variables
with open('/commonality_analysis/rs/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_rs = pickle.load(f)

with open('/commonality_analysis/rs/pls/unique_rs.pkl', 'rb') as f:
    unique_rs = pickle.load(f)

with open('/commonality_analysis/rs/pls/common_mh_rs.pkl', 'rb') as f:
    common_mh_rs = pickle.load(f)
# Set variables
for key in unique_mh_rs: # unique for MH
    globals()[key] = round(unique_mh_rs[key] * 100, 3)
for key in unique_rs: # unique for rs modalities
    globals()[key] = round(unique_rs[key] * 100, 3)
for key in common_mh_rs: # common for MH and rs modalities
    globals()[key] = round(common_mh_rs[key] * 100, 3)


# Define x
modalities = list(unique_mh_rs.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in common_mh_rs.keys()])
y2 = np.array([globals()[f'{key}'] for key in unique_mh_rs.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_rs.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1]

x_sorted = [base_rename_dict_rs.get(item, item) for item in np.array(x)[sorted_indices]] #checks if item exists in base_rename_dict, if item exists, it returns the corresponding value from the dictionary, if doesnt, it returns item itself

y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

# Plot
plt.figure(figsize=(6, 6))
plt.barh(x_sorted, y1_sorted, linewidth=1, color= '#f56f5c', alpha=0.6) #fill=False, edgecolor='darkgreen',
plt.barh(x_sorted, y2_sorted, left=y1_sorted, color='#B24745FF', linewidth=1, alpha=0.3)#fill=False,edgecolor='navy',
plt.barh(x_sorted, y3_sorted, left=y1_sorted+y2_sorted, color= '#79AF9799', linewidth=1, alpha=0.3)#fill=False,edgecolor='crimson',


for i, (value_c, value_u_mh, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    plt.text(value_c / 2, i, f'{value_c:.1f}%', ha='center', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh / 2, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh + value_u_mri / 2, i, f'{value_u_mri:.1f}%', ha='center', va='center', color='black', fontsize=10)


plt.xlabel("% Variance Explained", fontsize=25, labelpad=20)
#plt.legend(["Common Variance: Mental Health + rsMRI", "Unique variance: Mental Health", "Unique variance: rsMRI"], fontsize = 10, ncol=3, loc='lower center', bbox_to_anchor=(0.5, -0.32)) #, , bbox_to_anchor=(1.3, 0.98) loc='upper right'
#plt.title("Variance Attributed to Mental Health and MRI Modalities", fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=15)
plt.tick_params(axis='y', length=0)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(1.5))

plt.savefig("/CA-rsMRI.png",
            bbox_inches="tight", 
            pad_inches=1, 
            transparent=False, 
            facecolor="w", 
            edgecolor='w', 
            orientation='landscape',
            format='png')


plt.show()

## T1/T2

In [ ]:
# Rename phenotypes
base_rename_dict_struct = {"struct_fast":'FSL FAST',
"struct_sub_first":'FSL FIRST',
"struct_aseg_mean_intensity":'ASEG Mean Thickness',
"struct_aseg_volume":'ASEG Volume',
"struct_ba_exvivo_area":'BA ex-vivo Area',
"struct_ba_exvivo_mean_thickness":'BA ex-vivo Mean Thickness',
"struct_ba_exvivo_volume":'BA ex-vivo Volume',
"struct_a2009s_area":'aparc.a2009s Area',
"struct_a2009s_mean_thickness":'aparc.a2009s Mean Thickness',
"struct_a2009s_volume":'aparc.a2009s Volume',
"struct_dkt_area":'DKT Area',
"struct_dkt_mean_thickness":'DKT Mean Thickness',
"struct_dkt_volume":'DKT Volume',
"struct_desikan_gw":'Desikan GM/WM Intensity',
"struct_desikan_pial":'Desikan Pial',
"struct_desikan_white_area":'Desikan WM Area',
"struct_desikan_white_mean_thickness":'Desikan WM Mean Thickness',
"struct_desikan_white_volume":'Desikan WM Volume',
"struct_subsegmentation":'Subcortical Volumetric Subseg.',
'T1_T2_whole_brain':'Whole-brain T1/T2'}

In [ ]:
# Load variables
with open('/commonality_analysis/struct/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/unique_t1_t2.pkl', 'rb') as f:
    unique_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/common_mh_t1_t2.pkl', 'rb') as f:
    common_mh_t1_t2 = pickle.load(f)
# Set variables
for key in unique_mh_t1_t2: # unique for MH
    globals()[key] = round(unique_mh_t1_t2[key] * 100, 3)
for key in unique_t1_t2: # unique for rs modalities
    globals()[key] = round(unique_t1_t2[key] * 100, 3)
for key in common_mh_t1_t2: # common for MH and rs modalities
    globals()[key] = round(common_mh_t1_t2[key] * 100, 3)

# Define x
modalities = list(unique_mh_t1_t2.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in common_mh_t1_t2.keys()])
y2 = np.array([globals()[f'{key}'] for key in unique_mh_t1_t2.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_t1_t2.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1] 

x_sorted = [base_rename_dict_struct.get(item, item) for item in np.array(x)[sorted_indices]] 
y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

plt.figure(figsize=(5, 10))
plt.barh(x_sorted, y1_sorted, linewidth=1, color= '#f56f5c', alpha=0.6) 
plt.barh(x_sorted, y2_sorted, left=y1_sorted, color='#B24745FF', linewidth=1, alpha=0.3)
plt.barh(x_sorted, y3_sorted, left=y1_sorted+y2_sorted, color= '#DF8F4499', linewidth=1, alpha=0.3)


for i, (value_c, value_u_mh, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    plt.text(value_c / 2.2, i, f'{value_c:.1f}%', ha='left', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh / 2, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh + value_u_mri / 2, i, f'{value_u_mri:.1f}%', ha='center', va='center', color='black', fontsize=10)


plt.xlabel("% Variance Explained", fontsize=25, labelpad=20)
#plt.legend(["Common Variance: Mental Health + sMRI", "Unique variance: Mental Health", "Unique variance: sMRI"], fontsize = 10, ncol=3, loc='lower center', bbox_to_anchor=(0.5, -0.2))
plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.tick_params(axis='y', length=0)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(1.5))

plt.savefig("/CA-sMRI.png",
            bbox_inches="tight", 
            pad_inches=1, 
            transparent=False, 
            facecolor="w", 
            edgecolor='w', 
            orientation='landscape',
            format='png')
plt.show()

In [ ]:
# Load variables
with open('/commonality_analysis/struct/pls/unique_mh.pkl', 'rb') as f:
    unique_mh_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/unique_t1_t2.pkl', 'rb') as f:
    unique_t1_t2 = pickle.load(f)

with open('/commonality_analysis/struct/pls/common_mh_t1_t2.pkl', 'rb') as f:
    common_mh_t1_t2 = pickle.load(f)
# Set variables
for key in unique_mh_t1_t2: # unique for MH
    globals()[key] = round(unique_mh_t1_t2[key] * 100, 3)
for key in unique_t1_t2: # unique for rs modalities
    globals()[key] = round(unique_t1_t2[key] * 100, 3)
for key in common_mh_t1_t2: # common for MH and rs modalities
    globals()[key] = round(common_mh_t1_t2[key] * 100, 3)

# Define x
modalities = list(unique_mh_t1_t2.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in common_mh_t1_t2.keys()])
y2 = np.array([globals()[f'{key}'] for key in unique_mh_t1_t2.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_t1_t2.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1] 

x_sorted = [base_rename_dict_struct.get(item, item) for item in np.array(x)[sorted_indices]] 
y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

plt.figure(figsize=(5, 10))
plt.barh(x_sorted, y1_sorted, linewidth=1, color= '#f56f5c', alpha=0.6) 
plt.barh(x_sorted, y2_sorted, left=y1_sorted, color='#B24745FF', linewidth=1, alpha=0.3)
plt.barh(x_sorted, y3_sorted, left=y1_sorted+y2_sorted, color= '#DF8F4499', linewidth=1, alpha=0.3)


for i, (value_c, value_u_mh, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    plt.text(value_c / 2.2, i, f'{value_c:.1f}%', ha='left', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh / 2, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=10)
    plt.text(value_c + value_u_mh + value_u_mri / 2, i, f'{value_u_mri:.1f}%', ha='center', va='center', color='black', fontsize=10)


plt.xlabel("% Variance Explained", fontsize=25, labelpad=20)
plt.legend(["Common Variance: Mental Health + sMRI", "Unique variance: Mental Health", "Unique variance: sMRI"], fontsize = 10, ncol=3, loc='lower center', bbox_to_anchor=(0.5, -0.2))

plt.yticks(fontsize=15)
plt.xticks(fontsize=15)
plt.tick_params(axis='y', length=0)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(1.5))

plt.savefig("/CA-sMRI.png",
            bbox_inches="tight", 
            pad_inches=1, 
            transparent=False, 
            facecolor="w", 
            edgecolor='w', 
            orientation='landscape',
            format='png')

plt.show()

##  Calculate % of variance explained by MRI

#### DTI

In [ ]:
# DTI
commonality_metrics_dti_df = pd.DataFrame.from_dict(commonality_metrics_dti, orient='index')
commonality_metrics_dti_df['Var Exp MRI'] = commonality_metrics_dti_df['Common'].round(2) / (commonality_metrics_dti_df['Common'].round(2) + commonality_metrics_dti_df['Unique: Mental Health'].round(2))
commonality_metrics_dti_df['Var Exp MRI'] = (commonality_metrics_dti_df['Var Exp MRI'] * 100).round(2)
commonality_metrics_dti_df['Total Var'] = commonality_metrics_dti_df.iloc[:, :-1].sum(axis=1)
commonality_metrics_dti_df = commonality_metrics_dti_df.sort_values(by='Total Var', ascending = False)
commonality_metrics_dti_df.to_csv('/commonality_analysis/var_explained_dti_full.csv')
commonality_metrics_dti_df

In [ ]:
# Rename according to plots
commonality_metrics_dti_df_renamed = commonality_metrics_dti_df.rename(index=base_rename_dict_dti)
commonality_metrics_dti_df_renamed.to_csv('/commonality_analysis/var_explained_dti_full_renamed.csv')
commonality_metrics_dti_df_renamed.round(2)

#### RS

In [ ]:
# RS
commonality_metrics_rs_df = pd.DataFrame.from_dict(commonality_metrics_rs, orient='index')
commonality_metrics_rs_df['Var Exp MRI'] = commonality_metrics_rs_df['Common'].round(2) / (commonality_metrics_rs_df['Common'].round(2) + commonality_metrics_rs_df['Unique: Mental Health'].round(2))
commonality_metrics_rs_df['Var Exp MRI'] = (commonality_metrics_rs_df['Var Exp MRI'] * 100).round(2)
commonality_metrics_rs_df['Total Var'] = commonality_metrics_rs_df.iloc[:, :-1].sum(axis=1)
commonality_metrics_rs_df = commonality_metrics_rs_df.sort_values(by='Total Var', ascending = False)
commonality_metrics_rs_df.to_csv('/commonality_analysis/var_explained_rs_full.csv')
commonality_metrics_rs_df

In [ ]:
# Rename according to plots
commonality_metrics_rs_df_renamed = commonality_metrics_rs_df.rename(index=base_rename_dict_rs)
commonality_metrics_rs_df_renamed.to_csv('/commonality_analysis/var_explained_rs_full_renamed.csv')
commonality_metrics_rs_df_renamed.round(2)

#### T1/T2

In [ ]:
# T1/T2
commonality_metrics_struct_df = pd.DataFrame.from_dict(commonality_metrics_struct, orient='index')
commonality_metrics_struct_df['Var Exp MRI'] = commonality_metrics_struct_df['Common'].round(2) / (commonality_metrics_struct_df['Common'].round(2) + commonality_metrics_struct_df['Unique: Mental Health'].round(2))
commonality_metrics_struct_df['Var Exp MRI'] = (commonality_metrics_struct_df['Var Exp MRI'] * 100).round(2)
commonality_metrics_struct_df['Total Var'] = commonality_metrics_struct_df.iloc[:, :-1].sum(axis=1)
commonality_metrics_struct_df = commonality_metrics_struct_df.sort_values(by='Total Var', ascending = False)
commonality_metrics_struct_df.to_csv('/var_explained_struct_full.csv')
commonality_metrics_struct_df

In [ ]:
# Rename according to plots
commonality_metrics_struct_df_renamed = commonality_metrics_struct_df.rename(index=base_rename_dict_struct)
commonality_metrics_struct_df_renamed.to_csv('/commonality_analysis/var_explained_struct_full_renamed.csv')
commonality_metrics_struct_df_renamed.round(2)

# Combine all three CA PLSR stacked bar plots in one figure

In [ ]:
# Birght
fig, axs = plt.subplots(1, 3, figsize=(57, 40))
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

######################## DTI
modalities = list(unique_mh_dti.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in unique_mh_dti.keys()])
y2 = np.array([globals()[f'{key}'] for key in common_mh_dti.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_dti.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1] 

x_sorted = [base_rename_dict_dti.get(item, item) for item in np.array(x)[sorted_indices]] 
y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

bar_height = 0.7
axs[0].barh(x_sorted, y1_sorted, height=bar_height, linewidth=2, color= '#B24745FF', alpha=0.6) 
axs[0].barh(x_sorted, y2_sorted, height=bar_height, left=y1_sorted, color='#66CCEEFF', linewidth=1, alpha=0.9) #cbdae8
axs[0].barh(x_sorted, y3_sorted, height=bar_height, left=y1_sorted+y2_sorted, color= '#5C88DAFF', linewidth=1, alpha=0.9)

for i, (value_u_mh, value_c, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    axs[0].text(value_u_mh / 2, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=33)
    axs[0].text(value_u_mh + value_c / 2, i, f'{value_c:.1f}%', ha='center', va='center', color='black', fontsize=33)
    if i == 0 or i == 1:
        axs[0].text(value_u_mh + value_c + value_u_mri / 2 + 3.0, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif 2 <= i <= 6: #i in range(2, 9)
        axs[0].text(value_u_mh + value_c + value_u_mri / 2 + 2.4, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif 7 <= i <= 12:
        axs[0].text(value_u_mh + value_c + value_u_mri / 2 + 2.2, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif 13 <= i <= 15:
        axs[0].text(value_u_mh + value_c + value_u_mri / 2 + 1.9, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif 16 <= i <= 28:
        axs[0].text(value_u_mh + value_c + value_u_mri / 2 + 1.5, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif 29 <= i <= 36:
        axs[0].text(value_u_mh + value_c + value_u_mri / 2 + 1.3, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    else:
        axs[0].text(value_u_mh + value_c + value_u_mri / 2, i, f'{value_u_mri.round(1)}%', ha='center', va='center', color='black', fontsize=33)

axs[0].set_yticklabels([])
axs[0].set_yticklabels(x_sorted, fontsize=45)
axs[0].set_xticklabels(axs[0].get_xticks(), fontsize=50)
axs[0].tick_params(axis='y', length=0)

axs[0].spines['top'].set_visible(False)
axs[0].spines['right'].set_visible(False)
axs[0].spines['left'].set_visible(False)

######################## RS

modalities = list(unique_mh_rs.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in unique_mh_rs.keys()])
y2 = np.array([globals()[f'{key}'] for key in common_mh_rs.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_rs.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1]

x_sorted = [base_rename_dict_rs.get(item, item) for item in np.array(x)[sorted_indices]] 
y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

# Plot
bar_height = 0.8
axs[1].barh(x_sorted, y1_sorted, height=bar_height, linewidth=1, color= '#B24745FF', alpha=0.6)
axs[1].barh(x_sorted, y2_sorted, height=bar_height, left=y1_sorted, color='#79AF9799', linewidth=1, alpha=0.6)  #edded7 418D87FF
axs[1].barh(x_sorted, y3_sorted, height=bar_height, left=y1_sorted+y2_sorted, color= '#2C715FFF', linewidth=1, alpha=0.6)

for i, (value_u_mh, value_c, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    axs[1].text(value_u_mh / 2, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=33)
    axs[1].text(value_u_mh + value_c / 2, i, f'{value_c:.1f}%', ha='center', va='center', color='black', fontsize=33)

    if i == 0 or i == 1:
        axs[1].text(value_u_mh + value_c + value_u_mri / 2 + 3.5, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i == 2:
        axs[1].text(value_u_mh + value_c + value_u_mri / 2 + 2.2, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i in range (3,5): 
        axs[1].text(value_u_mh + value_c + value_u_mri / 2 + 2, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i in range (4,7):
        axs[1].text(value_u_mh + value_c + value_u_mri / 2 + 1.6, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    else:
        axs[1].text(value_u_mh + value_c + value_u_mri / 2, i, f'{value_u_mri.round(1)}%', ha='center', va='center', color='black', fontsize=33)

axs[1].set_yticklabels([])
axs[1].set_yticklabels(x_sorted, fontsize=52)
axs[1].set_xticklabels(axs[1].get_xticks(), fontsize=50)
axs[1].tick_params(axis='y', length=0)

axs[1].spines['top'].set_visible(False)
axs[1].spines['right'].set_visible(False)
axs[1].spines['left'].set_visible(False)

######################## T1/T2

modalities = list(unique_mh_t1_t2.keys())
x = [modality.replace('u_mh_from_', '') for modality in modalities]
y1 = np.array([globals()[f'{key}'] for key in unique_mh_t1_t2.keys()])
y2 = np.array([globals()[f'{key}'] for key in common_mh_t1_t2.keys()])
y3 = np.array([globals()[f'{key}'] for key in unique_t1_t2.keys()])

# Sort plots based on total variance
total_variance = y1 + y2 + y3
sorted_indices = np.argsort(total_variance)[::1] 

x_sorted = [base_rename_dict_struct.get(item, item) for item in np.array(x)[sorted_indices]] 
y1_sorted = y1[sorted_indices]
y2_sorted = y2[sorted_indices]
y3_sorted = y3[sorted_indices]

bar_height = 0.8
axs[2].barh(x_sorted, y1_sorted, height=bar_height, linewidth=1, color= '#B24745FF', alpha=0.6) 
axs[2].barh(x_sorted, y2_sorted, height=bar_height, left=y1_sorted, color='#EDD03EFF', linewidth=1, alpha=0.5) #E37D41FF
axs[2].barh(x_sorted, y3_sorted, height=bar_height, left=y1_sorted+y2_sorted, color= '#DF8F44FF', linewidth=1, alpha=0.7)

# Adjust % positions
for i, (value_u_mh, value_c, value_u_mri) in enumerate(zip(y1_sorted, y2_sorted, y3_sorted)):
    axs[2].text(value_u_mh / 2 + 0.1, i, f'{value_u_mh:.1f}%', ha='center', va='center', color='black', fontsize=33)
    axs[2].text(value_u_mh + value_c / 2, i, f'{value_c:.1f}%', ha='center', va='center', color='black', fontsize=33)

    if i == 0:
        axs[2].text(value_u_mh + value_c + value_u_mri / 2 + 3.1, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i in range(1,4):
        axs[2].text(value_u_mh + value_c + value_u_mri / 2 + 2.8, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i in range(4,13):
        axs[2].text(value_u_mh + value_c + value_u_mri / 2 + 2.5, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i == 13:
        axs[2].text(value_u_mh + value_c + value_u_mri / 2 + 2.4, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    elif i in range(14,19):
        axs[2].text(value_u_mh + value_c + value_u_mri / 2 + 1.8, i, f'{value_u_mri.round(1)}%', ha='right', va='center', color='black', fontsize=33)
    else:
        axs[2].text(value_u_mh + value_c + value_u_mri / 2, i, f'{value_u_mri.round(1)}%', ha='center', va='center', color='black', fontsize=33)

axs[2].set_yticklabels([])
axs[2].set_yticklabels(x_sorted, fontsize=52)
axs[2].set_xticklabels(axs[2].get_xticks(), fontsize=50)
axs[2].tick_params(axis='y', length=0)


axs[2].spines['top'].set_visible(False)
axs[2].spines['right'].set_visible(False)
axs[2].spines['left'].set_visible(False)

####

axs[0].tick_params(axis='y', pad=10)
axs[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda val, pos: f'{int(val)}')) 
axs[1].tick_params(axis='y', pad=10)
axs[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda val, pos: f'{int(val)}')) 
axs[2].tick_params(axis='y', pad=10)
axs[2].xaxis.set_major_formatter(ticker.FuncFormatter(lambda val, pos: f'{int(val)}'))

####

axs[0].set_title('dwMRI', fontsize=100, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#66CCEEFF', boxstyle='round', pad=0.1, linewidth=4), y=0.97, x=0.42)
axs[1].set_title('rsMRI', fontsize=100, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#79AF9799', boxstyle='round', pad=0.1, linewidth=4), y=0.97, x=0.42)
axs[2].set_title('sMRI', fontsize=100, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#EDD03EFF', boxstyle='round', pad=0.1, linewidth=4), y=0.97, x=0.43)


# Add common title and x-axis label
fig.text(0.5, -0.03, '% Variance Explained', ha='center', fontsize=80)

# Add common legend
colors = ['#66CCEEFF', '#79AF9799', '#EDD03EFF',
          '#B24745FF', '#5C88DAFF', '#2C715FFF', '#DF8F44FF']
alphas = [0.9, 0.9, 0.5, 0.6, 0.9, 0.6, 0.9]
labels = ['Common Variance: Mental Health + dwMRI', 'Common Variance: Mental Health + rsMRI', 'Common Variance: Mental Health + sMRI',
          'Unique variance: Mental Health', 'Unique variance: dwMRI', 'Unique variance: rsMRI', 'Unique variance: sMRI']
handles = [plt.Rectangle((0,0),1,1, color=color, alpha=alpha) for color, alpha in zip(colors, alphas)]

fig.legend(handles, labels, loc='lower center', ncol=3, fontsize=65, frameon=False, bbox_to_anchor=(0.55, -0.2))
fig.tight_layout() #rect=[0, 0, 1, 0.96]

plt.subplots_adjust(top=1.2)

plt.savefig("/media/hcs-sci-psy-narun/IBu/Articles-Conferences/Articles/eLife/figures/finset/Fig4b.pdf",
            bbox_inches ="tight", 
            #pad_inches = 0.5, 
            transparent = False, 
            facecolor ="w", 
            edgecolor ='w', 
            orientation ='landscape',
            format='pdf')

plt.show()

#######################################################

# Incorporate demographics: sex, age*sex, etc.

- U (i) = R2_ijk – R2_jk
- U (j) = R2_ijk – R2_ik
- U (k) = R2_ijk – R2_ij

- C (ij) = R2_ik + R2_jk – R2_k – R2_ijk
- C (ik) = R2_ij + R2_jk – R2_j – R2_ijk
- C (jk) = R2_ij + R2_ik – R2_i – R2_ijk
- C (ijk) = R2_i + R2_j + R2_k – R2_ij – R2_ik – R2_jk + R2_ijk

- U (mh) = R2_mh_rs_age – R2_rs_age
- U (rs) = R2_mh_rs_age – R2_mh_age
- U (age) = R2_mh_rs_age – R2_mh_rs

- C (mh_rs) = R2_mh_age + R2_rs_age – R2_age – R2_mh_rs_age
- C (mh_age) = R2_mh_rs + R2_rs_age – R2_rs – R2_mh_rs_age
- C (rs_age) = R2_mh_rs + R2_mh_age – R2_mh – R2_mh_rs_age
- C (mh_rs_age) = R2_mh + R2_rs + R2_age – R2_mh_rs – R2_mh_age – R2_rs_age + R2_mh_rs_age

In [ ]:
basic_confounds_min = pd.read_csv('/ML_DATASETS/basic_confounds_min.csv')[['Age when attended assessment centre', 'eid', 'Sex', 'Age2', 'Age*Sex', 'Age2*Sex']]
demo = pd.DataFrame(basic_confounds_min)
demo = demo.rename(columns={'Age when attended assessment centre': 'Age'})
demo.to_csv('/PLS/brain/stacking/all/demo.csv', index=False)
demo = pd.read_csv('/PLS/brain/stacking/all/demo.csv')
demo_var = ['Age', 'Sex', 'Age2', 'Age*Sex', 'Age2*Sex']

## RS

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = ["0", "1", "2", "3", "4"]
all_g_mh_rs_demo_concat = []
demo_var = ['Age', 'Sex', 'Age2', 'Age*Sex', 'Age2*Sex']

for fold in folds:
    g_pred_rs_idp_rf = pd.read_csv(f'/PLS/brain/stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_rs_idp_rf, on='eid').merge(demo, on='eid').drop(columns=['eid'])
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_rs_idp_ts_best_stack_test': 'g_pred_rs_idp'})
    all_g_mh_rs_demo_concat.append(all_g)
    all_g_mh_rs_demo = pd.concat(all_g_mh_rs_demo_concat, axis=0, ignore_index=True)
    all_g_mh_rs_demo.to_csv('/commonality_analysis/g_real_pred_mh_rs_demo.csv', index=False)
    
model = LinearRegression()
r2_mh = model.fit(all_g_mh_rs_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_rs_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_rs = model.fit(all_g_mh_rs_demo['g_pred_rs_idp'].values.reshape(-1, 1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_rs_demo['g_pred_rs_idp'].values.reshape(-1, 1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_demo = model.fit(all_g_mh_rs_demo[demo_var].values, all_g_mh_rs_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_rs_demo[demo_var].values, all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_mh_rs_demo = model.fit(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real']).score(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))

r2_mh_demo = model.fit(pd.concat([all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real']).score(pd.concat([all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_rs_demo = model.fit(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real']).score(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_mh_rs = model.fit(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo['g_pred_mh']], axis=1), all_g_mh_rs_demo['g_real']).score(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo['g_pred_mh']], axis=1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))

In [ ]:
all_g_mh_rs_demo = pd.read_csv('/commonality_analysis/g_real_pred_mh_rs_demo.csv')

r2_mh = commonality_analysis(all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo['g_real'])
r2_rs = commonality_analysis(all_g_mh_rs_demo['g_pred_rs_idp'].values.reshape(-1, 1), all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_demo = commonality_analysis(all_g_mh_rs_demo[demo_var], all_g_mh_rs_demo['g_real'].values.reshape(-1, 1))
r2_mh_rs_demo = commonality_analysis(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real'])

r2_mh_demo = commonality_analysis(pd.concat([all_g_mh_rs_demo['g_pred_mh'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real'])
r2_rs_demo = commonality_analysis(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo[demo_var]], axis=1), all_g_mh_rs_demo['g_real'])
r2_mh_rs = commonality_analysis(pd.concat([all_g_mh_rs_demo['g_pred_rs_idp'], all_g_mh_rs_demo['g_pred_mh']], axis=1), all_g_mh_rs_demo['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# i = mh, j = rs, k = demo

u_mh = r2_mh_rs_demo - r2_rs_demo
u_rs = r2_mh_rs_demo - r2_mh_demo
u_demo = r2_mh_rs_demo - r2_mh_rs

c_mh_rs = r2_mh_demo + r2_rs_demo - r2_demo - r2_mh_rs_demo
c_mh_demo = r2_mh_rs + r2_rs_demo - r2_rs - r2_mh_rs_demo
c_rs_demo = r2_mh_rs + r2_mh_demo - r2_mh - r2_mh_rs_demo
c_mh_rs_demo = r2_mh + r2_rs + r2_demo - r2_mh_rs - r2_mh_demo - r2_rs_demo + r2_mh_rs_demo


print("R squared for MH:", r2_mh.round(3)) # f"{r2_mh_m:.5f}")
print("R squared for RS", r2_rs.round(3))
print("R squared for demo", r2_demo.round(3))
print("R squared for MH, RS, and demo", r2_mh_rs_demo.round(3))

print('_______________')
print("Unique variance for MH:", u_mh.round(3)) #"{:.5f}".format(u_mh)) # "{:.5f}".format(u_mh)
print("Unique variance for RS:", u_rs.round(3))
print("Unique variance for demo:", u_demo.round(3))

print('_______________')
print("Common variance for MH and RS:", c_mh_rs.round(3))
print("Common variance for MH and demo:", c_mh_demo.round(3))
print("Common variance for RS and demo:", c_rs_demo.round(3))
print("Common variance for MH, RS, and demo:", c_mh_rs_demo.round(3))

In [ ]:
# Make up a data frame with commonality metrics
mh_rs_demo_commonality = pd.DataFrame([{'u_mh-rs_plus_demo': u_mh, 'u_rs-rs_plus_demo': u_rs, 'u_demo-rs_plus_demo': u_demo,
                                       'c_mh_and_rs-rs_plus_demo': c_mh_rs, 'c_mh_and_demo-rs_plus_demo':c_mh_demo, 'c_rs_and_demo-rs_plus_demo': c_rs_demo, 'c_mh_and_rs_and_demo-rs_plus_demo': c_mh_rs_demo}]).round(4)
mh_rs_demo_commonality.to_csv('/commonality_analysis/mh_rs_demo_commonality.csv', index=False)

## DTI

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = ["0", "1", "2", "3", "4"]
all_g_mh_dti_demo_concat = []

for fold in folds:
    g_pred_dti_rf = pd.read_csv(f'/PLS/brain/stacking/g/DTI_All_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    g_pred_mh = g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_dti_rf, on='eid').merge(demo, on='eid').drop(columns=['eid'])
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_dti_all_stack_test': 'g_pred_dti'})
    all_g_mh_dti_demo_concat.append(all_g)
    all_g_mh_dti_demo = pd.concat(all_g_mh_dti_demo_concat, axis=0, ignore_index=True)
    all_g_mh_dti_demo.to_csv('/commonality_analysis/g_real_pred_mh_dti-all_demo.csv', index=False)

model = LinearRegression()
r2_mh = model.fit(all_g_mh_dti_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_dti_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_dti = model.fit(all_g_mh_dti_demo['g_pred_dti'].values.reshape(-1, 1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_dti_demo['g_pred_dti'].values.reshape(-1, 1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_demo = model.fit(all_g_mh_dti_demo[demo_var].values, all_g_mh_dti_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_dti_demo[demo_var].values, all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_mh_dti_demo = model.fit(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real']).score(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))

r2_mh_demo = model.fit(pd.concat([all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real']).score(pd.concat([all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_dti_demo = model.fit(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real']).score(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_mh_dti = model.fit(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo['g_pred_mh']], axis=1), all_g_mh_dti_demo['g_real']).score(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo['g_pred_mh']], axis=1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))

In [ ]:
all_g_mh_dti_demo = pd.read_csv('/commonality_analysis/g_real_pred_mh_dti-all_demo.csv')
r2_mh = commonality_analysis(all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo['g_real'])
r2_dti = commonality_analysis(all_g_mh_dti_demo['g_pred_dti'].values.reshape(-1, 1), all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_demo = commonality_analysis(all_g_mh_dti_demo[demo_var], all_g_mh_dti_demo['g_real'].values.reshape(-1, 1))
r2_mh_dti_demo = commonality_analysis(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real'])

r2_mh_demo = commonality_analysis(pd.concat([all_g_mh_dti_demo['g_pred_mh'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real'])
r2_dti_demo = commonality_analysis(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo[demo_var]], axis=1), all_g_mh_dti_demo['g_real'])
r2_mh_dti = commonality_analysis(pd.concat([all_g_mh_dti_demo['g_pred_dti'], all_g_mh_dti_demo['g_pred_mh']], axis=1), all_g_mh_dti_demo['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# i = mh, j = rs, k = demo

u_mh = r2_mh_dti_demo - r2_dti_demo
u_dti = r2_mh_dti_demo - r2_mh_demo
u_demo = r2_mh_dti_demo - r2_mh_dti

c_mh_dti = r2_mh_demo + r2_dti_demo - r2_demo - r2_mh_dti_demo
c_mh_demo = r2_mh_dti + r2_dti_demo - r2_dti - r2_mh_dti_demo
c_dti_demo = r2_mh_dti + r2_mh_demo - r2_mh - r2_mh_dti_demo
c_mh_dti_demo = r2_mh + r2_dti + r2_demo - r2_mh_dti - r2_mh_demo - r2_dti_demo + r2_mh_dti_demo


print("R squared for MH:", r2_mh.round(3))
print("R squared for DTI All", r2_dti.round(3))
print("R squared for demo", r2_demo.round(3))
print("R squared for MH, DTI All, and demo", r2_mh_dti_demo.round(3))

print('_______________')
print("Unique variance for MH:", u_mh.round(3)) 
print("Unique variance for DTI All:", u_dti.round(3)) #f"{u_dti:.5f}".round(3))
print("Unique variance for demo:", u_demo.round(3))

print('_______________')
print("Common variance for MH and DTI All:", c_mh_dti.round(3))
print("Common variance for MH and demo:", c_mh_demo.round(3))
print("Common variance for dti and demo:", c_dti_demo.round(3))
print("Common variance for MH, DTI All, and demo:", c_mh_dti_demo.round(3))

In [ ]:
# Make up a data frame with commonality metrics
mh_dti_demo_commonality = pd.DataFrame([{'u_mh-dti_plus_demo': u_mh, 'u_dti-dti_plus_demo': u_dti, 'u_demo-dti_plus_demo': u_demo,
                                       'c_mh_and_dti-dti_plus_demo': c_mh_dti, 'c_mh_and_demo-dti_plus_demo':c_mh_demo,
                                       'c_dti_and_demo-dti_plus_demo': c_dti_demo, 'c_mh_and_dti_and_demo-dti_plus_demo': c_mh_dti_demo}]).round(4)
mh_dti_demo_commonality.to_csv('/commonality_analysis/mh_dti_demo_commonality.csv', index=False)

## T1/T2

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = ["0", "1", "2", "3", "4"]
all_g_mh_t1_demo_concat = []

for fold in folds:
    g_pred_t1_svr = pd.read_csv(f'/PLS/brain/stacking/g/T1_T2_whole_brain_target_pred_2nd_level_svr_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_t1_svr, on='eid').merge(demo, on='eid').drop(columns=['eid'])
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_mribest_stack_test': 'g_pred_t1'}) #the column was incorrectly named g_pred_mribest_stack_test instead of g_pred_t1_stack_test
    all_g_mh_t1_demo_concat.append(all_g)
    all_g_mh_t1_demo = pd.concat(all_g_mh_t1_demo_concat, axis=0, ignore_index=True)
    all_g_mh_t1_demo.to_csv('/commonality_analysis/g_real_pred_mh_t1_demo.csv', index=False)

model = LinearRegression()
r2_mh = model.fit(all_g_mh_t1_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_t1_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_t1 = model.fit(all_g_mh_t1_demo['g_pred_t1'].values.reshape(-1, 1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_t1_demo['g_pred_t1'].values.reshape(-1, 1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_demo = model.fit(all_g_mh_t1_demo[demo_var].values, all_g_mh_t1_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_t1_demo[demo_var].values, all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_mh_t1_demo = model.fit(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real']).score(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))

r2_mh_demo = model.fit(pd.concat([all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real']).score(pd.concat([all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_t1_demo = model.fit(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real']).score(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_mh_t1 = model.fit(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo['g_pred_mh']], axis=1), all_g_mh_t1_demo['g_real']).score(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo['g_pred_mh']], axis=1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))


In [ ]:
all_g_mh_t1_demo = pd.read_csv('/commonality_analysis/g_real_pred_mh_t1_demo.csv')
r2_mh = commonality_analysis(all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo['g_real'])
r2_t1 = commonality_analysis(all_g_mh_t1_demo['g_pred_t1'].values.reshape(-1, 1), all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_demo = commonality_analysis(all_g_mh_t1_demo[demo_var], all_g_mh_t1_demo['g_real'].values.reshape(-1, 1))
r2_mh_t1_demo = commonality_analysis(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real'])

r2_mh_demo = commonality_analysis(pd.concat([all_g_mh_t1_demo['g_pred_mh'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real'])
r2_t1_demo = commonality_analysis(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo[demo_var]], axis=1), all_g_mh_t1_demo['g_real'])
r2_mh_t1 = commonality_analysis(pd.concat([all_g_mh_t1_demo['g_pred_t1'], all_g_mh_t1_demo['g_pred_mh']], axis=1), all_g_mh_t1_demo['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# i = mh, j = rs, k = demo

u_mh = r2_mh_t1_demo - r2_t1_demo
u_t1 = r2_mh_t1_demo - r2_mh_demo
u_demo = r2_mh_t1_demo - r2_mh_t1

c_mh_t1 = r2_mh_demo + r2_t1_demo - r2_demo - r2_mh_t1_demo
c_mh_demo = r2_mh_t1 + r2_t1_demo - r2_t1 - r2_mh_t1_demo
c_t1_demo = r2_mh_t1 + r2_mh_demo - r2_mh - r2_mh_t1_demo
c_mh_t1_demo = r2_mh + r2_t1 + r2_demo - r2_mh_t1 - r2_mh_demo - r2_t1_demo + r2_mh_t1_demo


print("R squared for MH:", r2_mh.round(3))
print("R squared for T1", r2_t1.round(3))
print("R squared for demo", r2_demo.round(3))
print("R squared for MH, T1, and demo", r2_mh_t1_demo.round(3))

print('_______________')
print("Unique variance for MH:", u_mh.round(3)) 
print("Unique variance for T1:", u_t1.round(3)) #f"{u_t1:.5f}".round(3))
print("Unique variance for demo:", u_demo.round(3))

print('_______________')
print("Common variance for MH and T1:", c_mh_t1.round(3))
print("Common variance for MH and demo:", c_mh_demo.round(3))
print("Common variance for T1 and demo:", c_t1_demo.round(3))
print("Common variance for MH, T1, and demo:", c_mh_t1_demo.round(3))

In [ ]:
# Make up a data frame with commonality metrics
mh_t1_demo_commonality = pd.DataFrame([{'u_mh-t1_plus_demo': u_mh, 'u_t1-t1_plus_demo': u_t1, 'u_demo-t1_plus_demo': u_demo,
                                       'c_mh_and_t1-t1_plus_demo': c_mh_t1, 'c_mh_and_demo-t1_plus_demo':c_mh_demo, 'c_t1_and_demo-t1_plus_demo': c_t1_demo, 'c_mh_and_t1_and_demo-t1_plus_demo': c_mh_t1_demo}]).round(4)
mh_t1_demo_commonality.to_csv('/commonality_analysis/mh_t1_demo_commonality.csv', index=False)

## All modalities

In [ ]:
# Get R2 for each model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
folds = ["0", "1", "2", "3", "4"]
all_g_mh_mri_demo_concat = []

for fold in folds:
    g_pred_mri_all = pd.read_csv(f'/PLS/brain/stacking/g/All_modalities_target_pred_2nd_level_xgb_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_mri_all, on='eid').merge(demo, on='eid').drop(columns=['eid'])
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_mribest_stack_test': 'g_pred_mri'})
    all_g_mh_mri_demo_concat.append(all_g)
    all_g_mh_mri_demo = pd.concat(all_g_mh_mri_demo_concat, axis=0, ignore_index=True)
    all_g_mh_mri_demo.to_csv('/commonality_analysis/g_real_pred_mh_all-mri_demo.csv', index=False)


model = LinearRegression()
r2_mh = model.fit(all_g_mh_mri_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_mri_demo['g_pred_mh'].values.reshape(-1, 1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_mri = model.fit(all_g_mh_mri_demo['g_pred_mri'].values.reshape(-1, 1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_mri_demo['g_pred_mri'].values.reshape(-1, 1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_demo = model.fit(all_g_mh_mri_demo[demo_var], all_g_mh_mri_demo['g_real'].values.reshape(-1, 1)).score(all_g_mh_mri_demo[demo_var], all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_mh_mri_demo = model.fit(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real']).score(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))

r2_mh_demo = model.fit(pd.concat([all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real']).score(pd.concat([all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_mri_demo = model.fit(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real']).score(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_mh_mri = model.fit(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo['g_pred_mh']], axis=1), all_g_mh_mri_demo['g_real']).score(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo['g_pred_mh']], axis=1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))

In [ ]:
all_g_mh_mri_demo = pd.read_csv('/commonality_analysis/g_real_pred_mh_all-mri_demo.csv')

r2_mh = commonality_analysis(all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo['g_real'])
r2_mri = commonality_analysis(all_g_mh_mri_demo['g_pred_mri'].values.reshape(-1, 1), all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_demo = commonality_analysis(all_g_mh_mri_demo[demo_var], all_g_mh_mri_demo['g_real'].values.reshape(-1, 1))
r2_mh_mri_demo = commonality_analysis(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real'])

r2_mh_demo = commonality_analysis(pd.concat([all_g_mh_mri_demo['g_pred_mh'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real'])
r2_mri_demo = commonality_analysis(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo[demo_var]], axis=1), all_g_mh_mri_demo['g_real'])
r2_mh_mri = commonality_analysis(pd.concat([all_g_mh_mri_demo['g_pred_mri'], all_g_mh_mri_demo['g_pred_mh']], axis=1), all_g_mh_mri_demo['g_real'])

In [ ]:
# Calculate unique/common variance
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# i = mh, j = rs, k = demo

u_mh = r2_mh_mri_demo - r2_mri_demo
u_mri = r2_mh_mri_demo - r2_mh_demo
u_demo = r2_mh_mri_demo - r2_mh_mri

c_mh_mri = r2_mh_demo + r2_mri_demo - r2_demo - r2_mh_mri_demo
c_mh_demo = r2_mh_mri + r2_mri_demo - r2_mri - r2_mh_mri_demo
c_mri_demo = r2_mh_mri + r2_mh_demo - r2_mh - r2_mh_mri_demo
c_mh_mri_demo = r2_mh + r2_mri + r2_demo - r2_mh_mri - r2_mh_demo - r2_mri_demo + r2_mh_mri_demo

print("R squared for MH:", r2_mh.round(3)) 
print("R squared for MRI All (no tang)", r2_mri.round(3))
print("R squared for demo", r2_demo.round(3))
print("R squared for MH, MRI All (no tang), and demo", r2_mh_mri_demo.round(3))

print('_______________')
print("Unique variance for MH:", u_mh.round(3)) #"{:.5f}".format(u_mh)) # "{:.5f}".format(u_mh)
print("Unique variance for MRI All (no tang):", u_mri.round(3)) #f"{u_mri:.5f}")
print("Unique variance for demo:", u_demo.round(3))

print('_______________')
print("Common variance for MH and MRI All (no tang):", c_mh_mri.round(3))
print("Common variance for MH and demo:", c_mh_demo.round(3))
print("Common variance for MRI All (no tang) and demo:", c_mri_demo.round(3))
print("Common variance for MH, MRI All, and demo:", c_mh_mri_demo.round(3))


In [ ]:
# Make up a data frame with commonality metrics
mh_mri_demo_commonality = pd.DataFrame([{'u_mh-mri_plus_demo': u_mh, 'u_mri-mri_plus_demo': u_mri, 'u_demo-mri_plus_demo': u_demo,
                                       'c_mh_and_mri-mri_plus_demo': c_mh_mri, 'c_mh_and_demo-mri_plus_demo': c_mh_demo, 'c_mri_and_demo-mri_plus_demo': c_mri_demo, 'c_mh_and_mri_and_demo-mri_plus_demo': c_mh_mri_demo}]).round(4)
mh_mri_demo_commonality.to_csv('/commonality_analysis/mh_mri_demo_commonality.csv', index=False)

# Extract commonality metrics for demo sample

In [ ]:
# Get a table with commonality metrics
mh_mri_demo_commonality_path = '/commonality_analysis/mh_mri_demo_commonality.csv'
mh_rs_demo_commonality_path = '/commonality_analysis/mh_rs_demo_commonality.csv'
mh_t1_demo_commonality_path = '/commonality_analysis/mh_t1_demo_commonality.csv'
mh_dti_demo_commonality_path = '/commonality_analysis/mh_dti_demo_commonality.csv'

# Load the data from CSV files
mh_mri_demo_commonality = pd.read_csv(mh_mri_demo_commonality_path)
mh_rs_demo_commonality = pd.read_csv(mh_rs_demo_commonality_path)
mh_t1_demo_commonality = pd.read_csv(mh_t1_demo_commonality_path)
mh_dti_demo_commonality = pd.read_csv(mh_dti_demo_commonality_path)


In [ ]:
# ABC, Common to MH, MRI, and Demo
c_mh_and_mri_and_demo = (mh_mri_demo_commonality['c_mh_and_mri_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
c_mh_and_rs_and_demo  = (mh_rs_demo_commonality['c_mh_and_rs_and_demo-rs_plus_demo'].iloc[0]*100).round(2)
c_mh_and_t1_and_demo  = (mh_t1_demo_commonality['c_mh_and_t1_and_demo-t1_plus_demo'].iloc[0]*100).round(2)
c_mh_and_dti_and_demo  = (mh_dti_demo_commonality['c_mh_and_dti_and_demo-dti_plus_demo'].iloc[0]*100).round(2)

# AB, Common to MH and MRI
c_mh_and_mri = (mh_mri_demo_commonality['c_mh_and_mri-mri_plus_demo'].iloc[0]*100).round(2)
c_mh_and_rs  = (mh_rs_demo_commonality['c_mh_and_rs-rs_plus_demo'].iloc[0]*100).round(2)
c_mh_and_t1  = (mh_t1_demo_commonality['c_mh_and_t1-t1_plus_demo'].iloc[0]*100).round(2)
c_mh_and_dti  = (mh_dti_demo_commonality['c_mh_and_dti-dti_plus_demo'].iloc[0]*100).round(2)

# AC, Common to MH and Demo
c_mh_and_demo_mri = (mh_mri_demo_commonality['c_mh_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
c_mh_and_demo_rs  = (mh_rs_demo_commonality['c_mh_and_demo-rs_plus_demo'].iloc[0]*100).round(2)
c_mh_and_demo_t1  = (mh_t1_demo_commonality['c_mh_and_demo-t1_plus_demo'].iloc[0]*100).round(2)
c_mh_and_demo_dti  = (mh_dti_demo_commonality['c_mh_and_demo-dti_plus_demo'].iloc[0]*100).round(2)

# BC, Common to MRI and Demo
c_mri_and_demo = (mh_mri_demo_commonality['c_mri_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
c_rs_and_demo  = (mh_rs_demo_commonality['c_rs_and_demo-rs_plus_demo'].iloc[0]*100).round(2)
c_t1_and_demo  = (mh_t1_demo_commonality['c_t1_and_demo-t1_plus_demo'].iloc[0]*100).round(2)
c_dti_and_demo  = (mh_dti_demo_commonality['c_dti_and_demo-dti_plus_demo'].iloc[0]*100).round(2)

# A, Unique to MH
u_mh_mri = (mh_mri_demo_commonality['u_mh-mri_plus_demo'].iloc[0]*100).round(2)
u_mh_rs = (mh_rs_demo_commonality['u_mh-rs_plus_demo'].iloc[0]*100).round(2)
u_mh_t1 = (mh_t1_demo_commonality['u_mh-t1_plus_demo'].iloc[0]*100).round(2)
u_mh_dti = (mh_dti_demo_commonality['u_mh-dti_plus_demo'].iloc[0]*100).round(2)

# B, Unique to MRI
u_mri = (mh_mri_demo_commonality['u_mri-mri_plus_demo'].iloc[0]*100).round(2)
u_rs = (mh_rs_demo_commonality['u_rs-rs_plus_demo'].iloc[0]*100).round(2)
u_t1 = (mh_t1_demo_commonality['u_t1-t1_plus_demo'].iloc[0]*100).round(2)
u_dti = (mh_dti_demo_commonality['u_dti-dti_plus_demo'].iloc[0]*100).round(2)

# C, Unique to Demo
u_demo_mri = (mh_mri_demo_commonality['u_demo-mri_plus_demo'].iloc[0]*100).round(2)
u_demo_rs = (mh_rs_demo_commonality['u_demo-rs_plus_demo'].iloc[0]*100).round(2)
u_demo_t1 = (mh_t1_demo_commonality['u_demo-t1_plus_demo'].iloc[0]*100).round(2)
u_demo_dti = (mh_dti_demo_commonality['u_demo-dti_plus_demo'].iloc[0]*100).round(2)

In [ ]:
# Extract the required values and round them
data = {
    'Index': ['dti', 'rs', 't1', 'mri'],
    'Unique Mental Health': [
        (mh_dti_demo_commonality['u_mh-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['u_mh-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['u_mh-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['u_mh-mri_plus_demo'].iloc[0]*100).round(2)
    ],
    'Unique MRI': [
        (mh_dti_demo_commonality['u_dti-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['u_rs-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['u_t1-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['u_mri-mri_plus_demo'].iloc[0]*100).round(2)
    ],
    'Unique Demo': [
        (mh_dti_demo_commonality['u_demo-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['u_demo-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['u_demo-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['u_demo-mri_plus_demo'].iloc[0]*100).round(2)
    ],
    'Common to MH and MRI': [
        (mh_dti_demo_commonality['c_mh_and_dti-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['c_mh_and_rs-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['c_mh_and_t1-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['c_mh_and_mri-mri_plus_demo'].iloc[0]*100).round(2)
    ],
    'Common to MH and Demo': [
        (mh_dti_demo_commonality['c_mh_and_demo-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['c_mh_and_demo-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['c_mh_and_demo-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['c_mh_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
    ],
    'Common to MH, MRI, and Demo': [
        (mh_dti_demo_commonality['c_mh_and_dti_and_demo-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['c_mh_and_rs_and_demo-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['c_mh_and_t1_and_demo-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['c_mh_and_mri_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
    ],
    'Common to MRI and Demo': [
        (mh_dti_demo_commonality['c_dti_and_demo-dti_plus_demo'].iloc[0]*100).round(2),
        (mh_rs_demo_commonality['c_rs_and_demo-rs_plus_demo'].iloc[0]*100).round(2),
        (mh_t1_demo_commonality['c_t1_and_demo-t1_plus_demo'].iloc[0]*100).round(2),
        (mh_mri_demo_commonality['c_mri_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)
df.to_csv('/commonality_analysis/commonality_metrics_plus_demo_supplem_table.csv', index=False)
df

In [ ]:
import matplotlib.pyplot as plt

# Print default font settings
print("Default font family:", plt.rcParams['font.family'])
print("Default font size:", plt.rcParams['font.size'])

import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

# FINAL PLOT

In [ ]:
# Function to create scatter plot for each modality
def create_scatter_plot(ax, y_true, y_pred, title, cmap_name, edge_color, r_mean_stack, r_std_stack, r2_mean_stack, r2_std_stack, folds_data, modality):
    dist_i = np.sqrt((y_true - y_pred.mean())**2 + (y_pred - y_pred.mean())**2)
    sns.scatterplot(x=y_pred, y=y_true, c=dist_i, cmap=cmap_name, s=17, alpha=0.6, edgecolor=edge_color,  ax=ax, linewidth=0.1) #
    
    fontsize = 35
    
    # Add regression lines for each fold
    for i, fold in enumerate(folds):
        sns.regplot(x=folds_data[modality]['y_pred'][i], y=folds_data[modality]['y_true'][i], scatter=False, ax=ax, label=f'Fold {fold}', line_kws={"color": "red", "linewidth": 0.8})
    
    sns.despine(top=True, right=True, ax=ax)
    ax.set_xlabel('Predicted $g$-factor ($z$)', fontsize=fontsize)
    ax.set_ylabel('')
    ax.tick_params(axis='x', labelsize=30)
    ax.tick_params(axis='y', labelsize=30)
    ax.set_title(title, fontsize=30, y=1.3)
    
    ax.text(0.05, 1.05, f'$r_{{mean}}$ = {r_mean_stack:.2f} (SD={r_std_stack:.2f})', transform=ax.transAxes, fontsize=fontsize)
    if modality == 'rs':
        ax.text(0.05, 0.96, f'$R^2_{{mean}}$ = {r2_mean_stack:.3f} (SD={r2_std_stack:.2f})', transform=ax.transAxes, fontsize=fontsize)
    else:
        ax.text(0.05, 0.96, f'$R^2_{{mean}}$ = {r2_mean_stack:.2f} (SD={r2_std_stack:.2f})', transform=ax.transAxes, fontsize=fontsize)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(1.5))

In [ ]:
# Get a table with commonality metrics without demo
commonality_metrics_no_demo = pd.read_csv('/commonality_analysis/commonality_metrics_no_demo.csv')
# Extract the values from the DataFrame
u_mh_rs_na = commonality_metrics_no_demo['u_mh_rs_na'].iloc[0]
u_rs_na = commonality_metrics_no_demo['u_rs_na'].iloc[0]
c_mh_and_rs_na = commonality_metrics_no_demo['c_mh_and_rs_na'].iloc[0]
u_mh_dti_na = commonality_metrics_no_demo['u_mh_dti_na'].iloc[0]
u_dti_na = commonality_metrics_no_demo['u_dti_na'].iloc[0]
c_mh_and_dti_na = commonality_metrics_no_demo['c_mh_and_dti_na'].iloc[0]
u_mh_t1_na = commonality_metrics_no_demo['u_mh_t1_na'].iloc[0]
u_t1_na = commonality_metrics_no_demo['u_t1_na'].iloc[0].astype(float).round(3)
c_mh_and_t1_na = commonality_metrics_no_demo['c_mh_and_t1_na'].iloc[0]
u_mh_mri_na = commonality_metrics_no_demo['u_mh_mri_na'].iloc[0]
u_mri_na = commonality_metrics_no_demo['u_mri_na'].iloc[0]
c_mh_and_mri_na = commonality_metrics_no_demo['c_mh_and_mri_na'].iloc[0]

In [ ]:
# Get a table with commonality metrics for demo
mh_mri_demo_commonality_path = '/commonality_analysis/mh_mri_demo_commonality.csv'
mh_rs_demo_commonality_path = '/commonality_analysis/mh_rs_demo_commonality.csv'
mh_t1_demo_commonality_path = '/commonality_analysis/mh_t1_demo_commonality.csv'
mh_dti_demo_commonality_path = '/commonality_analysis/mh_dti_demo_commonality.csv'

# Load the data from CSV files
mh_mri_demo_commonality = pd.read_csv(mh_mri_demo_commonality_path)
mh_rs_demo_commonality = pd.read_csv(mh_rs_demo_commonality_path)
mh_t1_demo_commonality = pd.read_csv(mh_t1_demo_commonality_path)
mh_dti_demo_commonality = pd.read_csv(mh_dti_demo_commonality_path)

# Extract variances
# ABC, Common to MH, MRI, and Demo
c_mh_and_mri_and_demo = (mh_mri_demo_commonality['c_mh_and_mri_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
c_mh_and_rs_and_demo  = (mh_rs_demo_commonality['c_mh_and_rs_and_demo-rs_plus_demo'].iloc[0]*100).round(2)
c_mh_and_t1_and_demo  = (mh_t1_demo_commonality['c_mh_and_t1_and_demo-t1_plus_demo'].iloc[0]*100).round(2)
c_mh_and_dti_and_demo  = (mh_dti_demo_commonality['c_mh_and_dti_and_demo-dti_plus_demo'].iloc[0]*100).round(2)

# AB, Common to MH and MRI
c_mh_and_mri = (mh_mri_demo_commonality['c_mh_and_mri-mri_plus_demo'].iloc[0]*100).round(2)
c_mh_and_rs  = (mh_rs_demo_commonality['c_mh_and_rs-rs_plus_demo'].iloc[0]*100).round(2)
c_mh_and_t1  = (mh_t1_demo_commonality['c_mh_and_t1-t1_plus_demo'].iloc[0]*100).round(2)
c_mh_and_dti  = (mh_dti_demo_commonality['c_mh_and_dti-dti_plus_demo'].iloc[0]*100).round(2)

# AC, Common to MH and Demo
c_mh_and_demo_mri = (mh_mri_demo_commonality['c_mh_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
c_mh_and_demo_rs  = (mh_rs_demo_commonality['c_mh_and_demo-rs_plus_demo'].iloc[0]*100).round(2)
c_mh_and_demo_t1  = (mh_t1_demo_commonality['c_mh_and_demo-t1_plus_demo'].iloc[0]*100).round(2)
c_mh_and_demo_dti  = (mh_dti_demo_commonality['c_mh_and_demo-dti_plus_demo'].iloc[0]*100).round(2)

# BC, Common to MRI and Demo
c_mri_and_demo = (mh_mri_demo_commonality['c_mri_and_demo-mri_plus_demo'].iloc[0]*100).round(2)
c_rs_and_demo  = (mh_rs_demo_commonality['c_rs_and_demo-rs_plus_demo'].iloc[0]*100).round(2)
c_t1_and_demo  = (mh_t1_demo_commonality['c_t1_and_demo-t1_plus_demo'].iloc[0]*100).round(2)
c_dti_and_demo  = (mh_dti_demo_commonality['c_dti_and_demo-dti_plus_demo'].iloc[0]*100).round(2)

# A, Unique to MH
u_mh_mri = (mh_mri_demo_commonality['u_mh-mri_plus_demo'].iloc[0]*100).round(2)
u_mh_rs = (mh_rs_demo_commonality['u_mh-rs_plus_demo'].iloc[0]*100).round(2)
u_mh_t1 = (mh_t1_demo_commonality['u_mh-t1_plus_demo'].iloc[0]*100).round(2)
u_mh_dti = (mh_dti_demo_commonality['u_mh-dti_plus_demo'].iloc[0]*100).round(2)

# B, Unique to MRI
u_mri = (mh_mri_demo_commonality['u_mri-mri_plus_demo'].iloc[0]*100).round(2)
u_rs = (mh_rs_demo_commonality['u_rs-rs_plus_demo'].iloc[0]*100).round(2)
u_t1 = (mh_t1_demo_commonality['u_t1-t1_plus_demo'].iloc[0]*100).round(2)
u_dti = (mh_dti_demo_commonality['u_dti-dti_plus_demo'].iloc[0]*100).round(2)

# C, Unique to Demo
u_demo_mri = (mh_mri_demo_commonality['u_demo-mri_plus_demo'].iloc[0]*100).round(2)
u_demo_rs = (mh_rs_demo_commonality['u_demo-rs_plus_demo'].iloc[0]*100).round(2)
u_demo_t1 = (mh_t1_demo_commonality['u_demo-t1_plus_demo'].iloc[0]*100).round(2)
u_demo_dti = (mh_dti_demo_commonality['u_demo-dti_plus_demo'].iloc[0]*100).round(2)

In [ ]:
# Define y_true and y_pred for all modalities
from sklearn.metrics import r2_score
folds = ["0", "1", "2", "3", "4"]
y_pred_pooled = []
y_true_pooled = []

for i,fold in enumerate(folds):
    y_pred = pd.read_csv(f'/PLS/brain/stacking/g/DTI_All_target_pred_2nd_level_rf_test_fold_{fold}.csv')['g_pred_dti_all_stack_test']
    y_pred_pooled.append(y_pred)
    y_pred_pooled_df = pd.DataFrame(pd.concat(y_pred_pooled, ignore_index=True)).rename(columns={'g_pred_dti_all_stack_test': 'y_pred'})
    y_true = pd.read_csv(f'/PLS/brain/stacking/g/DTI_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    y_true_pooled.append(y_true)
    y_true_pooled_df = pd.DataFrame(pd.concat(y_true_pooled, ignore_index=True)).rename(columns={'g_real_scaled': 'y_true'})

y_true_dti = y_true_pooled_df.copy()['y_true']
y_pred_dti = y_pred_pooled_df.copy()['y_pred']

r2_scores = []
pearson_corrs = []
for fold in folds:
    y_pred_fold = pd.read_csv(f'/PLS/brain/stacking/g/DTI_All_target_pred_2nd_level_rf_test_fold_{fold}.csv')['g_pred_dti_all_stack_test']
    y_true_fold = pd.read_csv(f'/PLS/brain/stacking/g/DTI_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    r2 = r2_score(y_true_fold, y_pred_fold)
    r2_scores.append(r2)
    corr, _ = pearsonr(y_true_fold.squeeze(), y_pred_fold.squeeze())
    pearson_corrs.append(corr)
    
# Annotate the plot with Pearson correlation and R² score
#corr, p = pearsonr(y_true, y_pred)
r_mean_dti = np.mean(pearson_corrs)
r_std_dti = np.std(pearson_corrs)
r2_mean_dti = np.mean(r2_scores)
r2_std_dti = np.std(r2_scores)


############################################# RS
folds = ["0", "1", "2", "3", "4"]
y_pred_pooled = []
y_true_pooled = []

for i,fold in enumerate(folds):
    y_pred = pd.read_csv(f'/PLS/brain/stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_rf_test_fold_{fold}.csv')['g_pred_rs_idp_ts_best_stack_test']
    y_pred_pooled.append(y_pred)
    y_pred_pooled_df = pd.DataFrame(pd.concat(y_pred_pooled, ignore_index=True)).rename(columns={'g_pred_rs_idp_ts_best_stack_test': 'y_pred'})
    y_true = pd.read_csv(f'/PLS/brain/stacking/g/RS_Aug_ID_g/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    y_true_pooled.append(y_true)
    y_true_pooled_df = pd.DataFrame(pd.concat(y_true_pooled, ignore_index=True)).rename(columns={'g_real_scaled': 'y_true'})

y_true_rs = y_true_pooled_df.copy()['y_true']
y_pred_rs = y_pred_pooled_df.copy()['y_pred']

r2_scores = []
pearson_corrs = []
for fold in folds:
    y_pred_fold = pd.read_csv(f'/PLS/brain/stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_rf_test_fold_{fold}.csv')['g_pred_rs_idp_ts_best_stack_test']
    y_true_fold = pd.read_csv(f'/PLS/brain/stacking/g/RS_Aug_ID_g/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    r2 = r2_score(y_true_fold, y_pred_fold)
    r2_scores.append(r2)
    corr, _ = pearsonr(y_true_fold.squeeze(), y_pred_fold.squeeze())
    pearson_corrs.append(corr)

# Annotate the plot with Pearson correlation and R² score
#corr, p = pearsonr(y_true, y_pred)
r_mean_rs = np.mean(pearson_corrs)
r_std_rs = np.std(pearson_corrs)
r2_mean_rs = np.mean(r2_scores)
r2_std_rs = np.std(r2_scores)

################################### T1/T2
folds = ["0", "1", "2", "3", "4"]
y_pred_pooled = []
y_true_pooled = []

for i,fold in enumerate(folds):
    y_pred = pd.read_csv(f'/PLS/brain/stacking/g/T1_T2_whole_brain_target_pred_2nd_level_svr_test_fold_{fold}.csv')['g_pred_mribest_stack_test']
    y_pred_pooled.append(y_pred)
    y_pred_pooled_df = pd.DataFrame(pd.concat(y_pred_pooled, ignore_index=True)).rename(columns={'g_pred_mribest_stack_test': 'y_pred'})
    y_true = pd.read_csv(f'/PLS/brain/stacking/g/T1_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    y_true_pooled.append(y_true)
    y_true_pooled_df = pd.DataFrame(pd.concat(y_true_pooled, ignore_index=True)).rename(columns={'g_real_scaled': 'y_true'})

y_true_t1 = y_true_pooled_df.copy()['y_true']
y_pred_t1 = y_pred_pooled_df.copy()['y_pred']


pearson_corrs = []
for fold in folds:
    y_pred_fold = pd.read_csv(f'/PLS/brain/stacking/g/T1_T2_whole_brain_target_pred_2nd_level_svr_test_fold_{fold}.csv')['g_pred_mribest_stack_test']
    y_true_fold = pd.read_csv(f'/PLS/brain/stacking/g/T1_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    r2 = r2_score(y_true_fold, y_pred_fold)
    r2_scores.append(r2)
    corr, _ = pearsonr(y_true_fold.squeeze(), y_pred_fold.squeeze())
    pearson_corrs.append(corr)
    
# Annotate the plot with Pearson correlation and R² score
#corr, p = pearsonr(y_true, y_pred)
r_mean_t1 = np.mean(pearson_corrs)
r_std_t1 = np.std(pearson_corrs)
r2_mean_t1 = np.mean(r2_scores)
r2_std_t1 = np.std(r2_scores)

######################################## All
folds = ["0", "1", "2", "3", "4"]
y_pred_pooled = []
y_true_pooled = []

for i,fold in enumerate(folds):
    y_pred = pd.read_csv(f'/PLS/brain/stacking/g/All_modalities_target_pred_2nd_level_xgb_test_fold_{fold}.csv')['g_pred_mribest_stack_test']
    y_pred_pooled.append(y_pred)
    y_pred_pooled_df = pd.DataFrame(pd.concat(y_pred_pooled, ignore_index=True)).rename(columns={'g_pred_mribest_stack_test': 'y_pred'})
    y_true = pd.read_csv(f'/PLS/brain/stacking/g/Stacked_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    y_true_pooled.append(y_true)
    y_true_pooled_df = pd.DataFrame(pd.concat(y_true_pooled, ignore_index=True)).rename(columns={'g_real_scaled': 'y_true'})

y_true_stack = y_true_pooled_df.copy()['y_true']
y_pred_stack  = y_pred_pooled_df.copy()['y_pred']

r2_scores = []
pearson_corrs = []
for fold in folds:
    y_pred_fold = pd.read_csv(f'/PLS/brain/stacking/g/All_modalities_target_pred_2nd_level_xgb_test_fold_{fold}.csv')['g_pred_mribest_stack_test']
    y_true_fold = pd.read_csv(f'/PLS/brain/stacking/g/Stacked_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled']
    r2 = r2_score(y_true_fold, y_pred_fold)
    r2_scores.append(r2)
    corr, _ = pearsonr(y_true_fold.squeeze(), y_pred_fold.squeeze())
    pearson_corrs.append(corr)
    
# Annotate the plot with Pearson correlation and R² score
#corr, p = pearsonr(y_true, y_pred)
r_mean_stack = np.mean(pearson_corrs)
r_std_stack = np.std(pearson_corrs)
r2_mean_stack = np.mean(r2_scores)
r2_std_stack = np.std(r2_scores)

In [ ]:
# Organize data to plot all scatterplots in one figure
folds = ['0', '1', '2', '3', '4']
folds_data = {
    'dti': {'y_pred': [], 'y_true': []},
    'rs': {'y_pred': [], 'y_true': []},
    't1t2': {'y_pred': [], 'y_true': []},
    'all': {'y_pred': [], 'y_true': []}
}

# DTI
for fold in folds:
    y_pred_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/DTI_All_target_pred_2nd_level_rf_test_fold_{fold}.csv')['g_pred_dti_all_stack_test'])
    y_pred_fold.columns = ['y_pred']
    y_true_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/DTI_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled'])
    y_true_fold.columns = ['y_true']
    folds_data['dti']['y_pred'].append(y_pred_fold)
    folds_data['dti']['y_true'].append(y_true_fold)

# RS
for fold in folds:
    y_pred_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_rf_test_fold_{fold}.csv')['g_pred_rs_idp_ts_best_stack_test'])
    y_pred_fold.columns = ['y_pred']
    y_true_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/RS_Aug_ID_g/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled'])
    y_true_fold.columns = ['y_true']
    folds_data['rs']['y_pred'].append(y_pred_fold)
    folds_data['rs']['y_true'].append(y_true_fold)

# T1/T2
for fold in folds:
    y_pred_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/T1_T2_whole_brain_target_pred_2nd_level_svr_test_fold_{fold}.csv')['g_pred_mribest_stack_test'])
    y_pred_fold.columns = ['y_pred']
    y_true_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/T1_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled'])
    y_true_fold.columns = ['y_true']
    folds_data['t1t2']['y_pred'].append(y_pred_fold)
    folds_data['t1t2']['y_true'].append(y_true_fold)

# All
for fold in folds:
    y_pred_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/All_modalities_target_pred_2nd_level_xgb_test_fold_{fold}.csv')['g_pred_mribest_stack_test'])
    y_pred_fold.columns = ['y_pred']
    y_true_fold = pd.DataFrame(pd.read_csv(f'/PLS/brain/stacking/g/Stacked_Aug_ID/g/target_real_test_feature_matched_scaled_fold_{fold}.csv')['g_real_scaled'])
    y_true_fold.columns = ['y_true']
    folds_data['all']['y_pred'].append(y_pred_fold)
    folds_data['all']['y_true'].append(y_true_fold)

In [ ]:
# Plot scatterplots and commonality analysis together
fig = plt.figure(figsize=(22, 24))

data_venn2 = [
    (u_mh_dti_na.round(3), u_dti_na.round(3), c_mh_and_dti_na.round(3), '#B24745FF', '#00A1D599', 'dwMRI', "$g$ ~"),
    (u_mh_rs_na.round(3), u_rs_na.round(3), c_mh_and_rs_na.round(3), '#B24745FF', '#79AF9799', 'rsMRI', "$g$ ~"),
    (u_mh_t1_na.round(3), u_t1_na.round(3), c_mh_and_t1_na.round(3), '#B24745FF', '#DF8F4499', 'sMRI', "$g$ ~"),
    (u_mh_mri_na.round(3), u_mri_na.round(3), c_mh_and_mri_na.round(3), '#B24745FF', '#6A659999', 'all MRI Stacked', "$g$ ~")
]


# Rearranged data for venn3
data_venn3 = [
    ((u_mh_dti.round(3), u_dti.round(3), c_mh_and_dti.round(3), u_demo_dti.round(3), c_mh_and_demo_dti.round(3), c_dti_and_demo.round(3), c_mh_and_dti_and_demo.round(3)),
     ('#B24745FF', '#00A1D599', '#374E5599'), ('Mental Health', 'dwMRI', 'Age and Sex')),
    ((u_mh_rs.round(3), u_rs.round(3), c_mh_and_rs.round(3), u_demo_rs.round(3), c_mh_and_demo_rs.round(3), c_rs_and_demo.round(3), c_mh_and_rs_and_demo.round(3)),
     ('#B24745FF', '#79AF9799', '#374E5599'), ('Mental Health', 'rsMRI', 'Age and Sex')),
    ((u_mh_t1.round(3), u_t1.round(3), c_mh_and_t1.round(3), u_demo_t1.round(3), c_mh_and_demo_t1.round(3), c_t1_and_demo.round(3), c_mh_and_t1_and_demo.round(3)),
     ('#B24745FF', '#DF8F4499', '#374E5599'), ('Mental Health', 'T1w/T2w MRI', 'Age and Sex')),
    ((u_mh_mri.round(3), u_mri.round(3), c_mh_and_mri.round(3), u_demo_mri.round(3), c_mh_and_demo_mri.round(3), c_mri_and_demo.round(3), c_mh_and_mri_and_demo.round(3)),
     ('#B24745FF', '#6A659999', '#374E5599'), ('Mental Health', 'all MRI Stacked', 'Age&Sex'))
]
 
# Define the positions of each subplot
positions = [
    [0.05, 0.76, 0.28, 0.15], [0.45, 0.73, 0.28, 0.2], [0.73, 0.73, 0.25, 0.2],
    [0.05, 0.505 , 0.28, 0.15], [0.45, 0.49 , 0.28, 0.2], [0.73, 0.49, 0.25, 0.2],
    [0.05, 0.25, 0.28, 0.15], [0.45, 0.24, 0.28, 0.2], [0.73, 0.22, 0.25, 0.2],
    [0.05, -0.004, 0.28, 0.15], [0.45, -0.013, 0.28, 0.2], [0.73, -0.013, 0.25, 0.2]
]

# Create subplots at the specified positions
axes = []
for pos in positions:
    axes.append(fig.add_axes(pos))
    
# Plot scatter plots on the left side
dticmap = sns.light_palette("#8cd0e5", as_cmap=True)
rscmap = sns.light_palette("#39ab7e", as_cmap=True)
t1cmap = sns.light_palette("#f8b976", as_cmap=True)
stackcmap = sns.light_palette("#826b88", as_cmap=True)

create_scatter_plot(axes[0], y_true_dti, y_pred_dti, '', dticmap, '#8cd0e5',
                    r_mean_dti, r_std_dti, r2_mean_dti, r2_std_dti,
                    folds_data, 'dti')
axes[0].set_xlim(-2, 2)
create_scatter_plot(axes[3], y_true_rs, y_pred_rs, '', rscmap, "#39ab7e",
                    r_mean_rs, r_std_rs, r2_mean_rs, r2_std_rs,
                    folds_data, 'rs')
axes[3].set_xlim(-2, 2)
create_scatter_plot(axes[6], y_true_t1, y_pred_t1, '', t1cmap, "#f8b976",
                    r_mean_t1, r_std_t1, r2_mean_t1, r2_std_t1,
                    folds_data, 't1t2')
axes[6].set_xlim(-2, 2)
create_scatter_plot(axes[9], y_true_stack, y_pred_stack, '', stackcmap, "#826b88",
                    r_mean_stack, r_std_stack, r2_mean_stack, r2_std_stack,
                    folds_data, 'all')
axes[9].set_xlim(-2, 2)

# Plot Venn diagrams on the right side
for i in range(4):

    # Venn diagrams for venn2
    u_mh_dti_na, u_dti_na, c_mh_and_dti_na, color1, color2, label, title = data_venn2[i]
    subset_sizes = (u_mh_dti_na, u_dti_na, c_mh_and_dti_na)
    colors = (color1, color2)
    venn = venn2(subsets=subset_sizes, set_labels=('', ''), set_colors=colors, ax=axes[i*3+1])

    # Apply transparency and set line width
    alphas = (0.20, 0.20)
    for patch, alpha in zip(venn.patches, alphas):
        if patch:
            patch.set_alpha(alpha)
            patch.set_edgecolor('white')
            patch.set_linewidth(2)
            patch.set_path_effects([path_effects.withStroke(linewidth=3, foreground='grey')])
            
    # Set font size for labels
    for text in venn.set_labels:
        text.set_fontsize(24)
    for text in venn.subset_labels:
        if text:
            text.set_fontsize(24)
            text.set_text(f'{text.get_text()}%')

    # Move annotations within the circles
    lab = venn.get_label_by_id('10') #left
    if lab:
        x, y = lab.get_position()
        lab.set_position((x + 0.03, y))

    lab = venn.get_label_by_id('01') #middle
    if lab:
        x, y = lab.get_position()
        lab.set_position((x - 0.01, y))

    # Adjust position of 'g~' in legend
    if i == 0:
        axes[i*3+1].set_title(title, fontsize=40, pad=3, y=1.0387, x=-0.35) # 1st
    elif i == 1:
        axes[i*3+1].set_title(title, fontsize=40, pad=3, y=1.105, x=-0.35) # 2nd
    elif i == 3:
        axes[i*3+1].set_title(title, fontsize=40, pad=3, y=0.98, x=-0.35) # the last legend
    else:
        # Keep the rest of the legends as they are
        axes[i*3+1].set_title(title, fontsize=40, pad=3, y=1.095, x=-0.35)

    # Create custom legend for each subplot
    handles = [
        patches.Patch(color=color1, alpha=0.3, label='Mental Health'),
        patches.Patch(color=color2, alpha=0.3, label=label),
        patches.Patch(color='#374E5599', alpha=0.3, label='Age&Sex')
    ]
    # Add legend elements
    if i == 0:
        axes[i*3+1].legend(handles=handles, loc='upper left', bbox_to_anchor=(-0.3, 1.23), ncol=3, fontsize=35, title='', frameon=False, columnspacing=0.5) # the first legend
    elif i == 1:
        axes[i*3+1].legend(handles=handles, loc='upper left', bbox_to_anchor=(-0.3, 1.31), ncol=3, fontsize=35, title='', frameon=False, columnspacing=0.5) # second row legend
    elif i == 2:
        axes[i*3+1].legend(handles=handles, loc='upper left', bbox_to_anchor=(-0.3, 1.30), ncol=3, fontsize=35, title='', frameon=False, columnspacing=0.5) # third row legend
    elif i == 3:
        axes[i*3+1].legend(handles=handles, loc='upper left', bbox_to_anchor=(-0.3, 1.16), ncol=3, fontsize=35, title='', frameon=False, columnspacing=0.5) # the last legend

    #axes[i*3+1].set_aspect(0.7)

for i in range(4):

    # Venn diagrams for venn3
    subset_sizes, colors, labels = data_venn3[i]
    venn = venn3(subsets=subset_sizes, set_labels=['', '', ''], set_colors=colors, ax=axes[i*3+2])

    # Apply transparency and set line width
    alphas = (0.20, 0.20, 0.20)
    for patch, alpha in zip(venn.patches, alphas):
        if patch:
            patch.set_alpha(alpha)
            patch.set_edgecolor('white')
            patch.set_linewidth(2)
            patch.set_path_effects([path_effects.withStroke(linewidth=3, foreground='grey')])

            #t = transforms.Affine2D().rotate_deg(45) + axes[i*3+2].transData
            #patch.set_transform(t)

    # Set font size for labels
    for text in venn.set_labels:
        text.set_fontsize(24)
    for text in venn.subset_labels:
        if text:
            text.set_fontsize(24)
            text.set_text(f'{text.get_text()}%')

    # Move annotations within the circles
    lab = venn.get_label_by_id('100') #left
    if lab:
        x, y = lab.get_position()
        lab.set_position((x + 0.05, y))

        lab = venn.get_label_by_id('010') #left
    if lab:
        x, y = lab.get_position()
        lab.set_position((x - 0.01, y))

# Add letters to subplots
labels = ['a', 'e', 'i', 'b', 'f', 'j', 'c', 'g', 'k', 'd', 'h', 'l']
positions = [
    (-0.4, 1.09), (-0.0, 0.98), (-0.0, 0.97),
    (-0.4, 1.13), (-0.0, 1.0), (-0.0, 0.92),
    (-0.4, 1.13), (-0.0, 0.985), (-0.0, 1.05),
    (-0.4, 1.17), (-0.0, 0.94), (-0.0, 0.938)
]

for i in range(len(axes)):
    axes[i].text(positions[i][0], positions[i][1], labels[i], transform=axes[i].transAxes,
                 fontsize=50,  fontweight='bold', va='top', ha='right')
#plt.subplots_adjust(wspace=0.05, hspace=1.8)
plt.tight_layout()  # Adjust the rect parameter to fit the legends more tightly rect=[0, 0, 1, 1]

# Common y axis for scatterplots
fig.text(-0.04, 0.5, '$g$-factor derived from ESEM ($z$)', va='center', rotation='vertical', fontsize=45)
#fig.text(0.35, 0.8, 'dwMRI Stacked', ha='center', fontsize=20, bbox=dict(facecolor='none', edgecolor='#826b88', boxstyle='round', pad=0.5)) #fontweight='bold',

axes[0].set_title('dwMRI Stacked', fontsize=33, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#8cd0e5', boxstyle='round', pad=0.2), y=1.242, x=0.42)
axes[3].set_title('rsMRI Stacked', fontsize=33, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#39ab7e', boxstyle='round', pad=0.2), y=1.25, x=0.41)
axes[6].set_title('sMRI Stacked', fontsize=33, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#f8b976', boxstyle='round', pad=0.2), y=1.258, x=0.40)
axes[9].set_title('all MRI Stacked', fontsize=33, fontweight='bold', bbox=dict(facecolor='none', edgecolor='#826b88', boxstyle='round', pad=0.2), y=1.25, x=0.45)

plt.savefig("/media/hcs-sci-psy-narun/IBu/Plots_and_Tables/final/svg/Fig5.svg",
            bbox_inches="tight", 
            pad_inches=1, 
            transparent=False, 
            facecolor="w", 
            edgecolor='w', 
            orientation='landscape',
            format='svg')

plt.show()

# Compute correlation between PLSR performance and % of variance explained

In [ ]:
# Commonality metrics PLS
commonality_metrics_struct_pls = pd.read_csv('/commonality_analysis/var_explained_struct_full_renamed.csv').rename(columns={'Unnamed: 0':'Modality'})[['Modality', 'Var Exp MRI']]
commonality_metrics_rs_pls = pd.read_csv('/commonality_analysis/var_explained_rs_full_renamed.csv').rename(columns={'Unnamed: 0':'Modality'})[['Modality', 'Var Exp MRI']]
commonality_metrics_dti_pls = pd.read_csv('/commonality_analysis/var_explained_dti_full_renamed.csv').rename(columns={'Unnamed: 0':'Modality'})[['Modality', 'Var Exp MRI']]

# Commonality metrics stack
commonality_metrics_stack = pd.read_csv('/PLS/brain/commonality_analysis/commonality_metrics_stack.csv')[['Modality', 'Var Exp MRI']]

## Model performance
# Model performance PLS sMRI
struct_perf_pls = pd.read_csv('/PLS/brain/performance/struct_perf_pls_mean.csv')[['Modality', 'R2', 'Pearson r']]
# Model performance PLS RS
rs_perf_pls = pd.read_csv('/PLS/brain/performance/rs_perf_pls_mean.csv')[['Modality', 'R2', 'Pearson r']]
# Model performance PLS DTI
dti_perf_pls = pd.read_csv('/PLS/brain/performance/dti_perf_pls_mean.csv')[['Modality', 'R2', 'Pearson r']]


# Model performance stack
rs_perf_stack = pd.read_csv('/PLS/brain/stacking/RS_IDP_Timeseries_best_metrics_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
rs_perf_stack_rf = rs_perf_stack[rs_perf_stack['Algorithm']=='rf']
rs_perf_stack_rf['Modality'] = 'rsMRI'
rs_perf_stack_rf = rs_perf_stack_rf[['Modality', 'R2', 'Pearson r']]

dti_perf_stack = pd.read_csv('/PLS/brain/stacking/DTI_All_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
dti_perf_stack_rf = dti_perf_stack[dti_perf_stack['Algorithm']=='rf']
dti_perf_stack_rf['Modality'] = 'dwMRI'
dti_perf_stack_rf = dti_perf_stack_rf[['Modality', 'R2', 'Pearson r']]

struct_perf_stack = pd.read_csv('/PLS/brain/stacking/T1_T2_whole_brain_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
struct_perf_stack_svr = struct_perf_stack[struct_perf_stack['Algorithm']=='svr']
struct_perf_stack_svr['Modality'] = 'T1w/T2w MRI'
struct_perf_stack_svr = struct_perf_stack_svr[['Modality', 'R2', 'Pearson r']]

all_mod_perf_stack = pd.read_csv('/PLS/brain/stacking/All_modalities_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
all_mod_perf_stack = all_mod_perf_stack[all_mod_perf_stack['Algorithm']=='xgb']
all_mod_perf_stack['Modality'] = 'Stacked MRI'
all_mod_perf_stack = all_mod_perf_stack[['Modality', 'R2', 'Pearson r']]

## Merge results of predicion and commonality analysis
# Stacked
stack_perf = pd.concat([rs_perf_stack_rf, dti_perf_stack_rf, struct_perf_stack_svr, all_mod_perf_stack], axis=0)
stack_res = pd.merge(stack_perf, commonality_metrics_stack, on = 'Modality')

In [ ]:
# Commonality metrics PLS
commonality_metrics_struct_pls = pd.read_csv('/commonality_analysis/var_explained_struct_full_renamed.csv').rename(columns={'Unnamed: 0':'Modality'})[['Modality', 'Var Exp MRI']]
commonality_metrics_rs_pls = pd.read_csv('/commonality_analysis/var_explained_rs_full_renamed.csv').rename(columns={'Unnamed: 0':'Modality'})[['Modality', 'Var Exp MRI']]
commonality_metrics_dti_pls = pd.read_csv('/commonality_analysis/var_explained_dti_full_renamed.csv').rename(columns={'Unnamed: 0':'Modality'})[['Modality', 'Var Exp MRI']]

# Commonality metrics stack
commonality_metrics_stack = pd.read_csv('/PLS/brain/commonality_analysis/commonality_metrics_stack.csv')[['Modality', 'Var Exp MRI']]

## Model performance
# Model performance PLS sMRI
struct_perf_pls = pd.read_csv('/PLS/brain/performance/struct_perf_pls_mean.csv')[['Modality', 'R2', 'Pearson r']]
# Model performance PLS RS
rs_perf_pls = pd.read_csv('/PLS/brain/performance/rs_perf_pls_mean.csv')[['Modality', 'R2', 'Pearson r']]
# Model performance PLS DTI
dti_perf_pls = pd.read_csv('/PLS/brain/performance/dti_perf_pls_mean.csv')[['Modality', 'R2', 'Pearson r']]


# Model performance stack
rs_perf_stack = pd.read_csv('/PLS/brain/stacking/RS_IDP_Timeseries_best_metrics_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
rs_perf_stack_rf = rs_perf_stack[rs_perf_stack['Algorithm']=='rf']
rs_perf_stack_rf['Modality'] = 'rsMRI'
rs_perf_stack_rf = rs_perf_stack_rf[['Modality', 'R2', 'Pearson r']]

dti_perf_stack = pd.read_csv('/PLS/brain/stacking/DTI_All_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
dti_perf_stack_rf = dti_perf_stack[dti_perf_stack['Algorithm']=='rf']
dti_perf_stack_rf['Modality'] = 'dwMRI'
dti_perf_stack_rf = dti_perf_stack_rf[['Modality', 'R2', 'Pearson r']]

struct_perf_stack = pd.read_csv('/PLS/brain/stacking/T1_T2_whole_brain_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
struct_perf_stack_svr = struct_perf_stack[struct_perf_stack['Algorithm']=='svr']
struct_perf_stack_svr['Modality'] = 'T1w/T2w MRI'
struct_perf_stack_svr = struct_perf_stack_svr[['Modality', 'R2', 'Pearson r']]

all_mod_perf_stack = pd.read_csv('/PLS/brain/stacking/All_modalities_stacked_five_folds.csv')[['Algorithm', 'MSE', 'MAE', 'R2', 'Pearson r']].groupby(['Algorithm']).mean().round(3).reset_index().sort_values(by='Pearson r', ascending=False)
all_mod_perf_stack = all_mod_perf_stack[all_mod_perf_stack['Algorithm']=='xgb']
all_mod_perf_stack['Modality'] = 'Stacked MRI'
all_mod_perf_stack = all_mod_perf_stack[['Modality', 'R2', 'Pearson r']]

## Merge results of predicion and commonality analysis
# Stacked
stack_perf = pd.concat([rs_perf_stack_rf, dti_perf_stack_rf, struct_perf_stack_svr, all_mod_perf_stack], axis=0)
stack_res = pd.merge(stack_perf, commonality_metrics_stack, on = 'Modality')

In [ ]:
# Rename PLS results to match commonality
# Struct
struct_rename_dict = {
    "Subcortical volumetric subsegmentation": "Subcortical Volumetric Subseg.",
    "ASEG volume": "ASEG Volume",
    "FSL FAST": "FSL FAST",
    "Whole-brain T1/T2": "Whole-brain T1/T2",
    "ASEG mean intensity": "ASEG Mean Thickness",
    "Desikan grey/white matter intensity": "Desikan GM/WM Intensity",
    "aparc a2009s volume": "aparc.a2009s Volume",
    "aparc a2009s mean thickness": "aparc.a2009s Mean Thickness",
    "Desikan-Killiany-Tourville volume": "DKT Volume",
    "Desikan white matter mean thickness": "Desikan WM Mean Thickness",
    "Desikan-Killiany-Tourville mean thickness": "DKT Mean Thickness",
    "aparc a2009s area": "aparc.a2009s Area",
    "Desikan white matter volume": "Desikan WM Volume",
    "FSL FIRST": "FSL FIRST",
    "Desikan pial": "Desikan Pial",
    "BA ex-vivo volume": "BA ex-vivo Volume",
    "BA ex-vivo mean thickness": "BA ex-vivo Mean Thickness",
    "Desikan-Killiany-Tourville area": "DKT Area",
    "Desikan white matter area": "Desikan WM Area",
    "BA ex-vivo area": "BA ex-vivo Area"
}

struct_perf_pls["Modality"] = struct_perf_pls["Modality"].map(struct_rename_dict)

# RS
rs_rename_dict = {
    "Tangent matrices 55 IC": "55 IC Func. Connectivity",
    "Schaefer7n200p MSA I Full correlation": "Schaefer200-I Func. Connectivity",
    "Schaefer7n500p MSA IV Full correlation": "Schaefer500-IV Func. Connectivity",
    "Glasser MSA I Full correlation": "Glasser-I Func. Connectivity",
    "Tangent matrices 21 IC": "21 IC Func. Connectivity",
    "Glasser MSA IV Full correlation": "Glasser-IV Func. Connectivity",
    "aparc MSA I Full correlation": "aparc-I Func. Connectivity",
    "aparc a2009s MSA I Full correlation": "aparc.a2009s-I Func. Connectivity",
    "Amplitudes 55 IC": "55 IC Amplitudes",
    "Amplitudes 21 IC": "21 IC Amplitudes"
}
rs_perf_pls["Modality"] = rs_perf_pls["Modality"].map(rs_rename_dict)

# DTI
dti_rename_dict = {
    "aparc MSA I Connectome Streamline Count": "aparc-I Streamline Count",
    "aparc MSA I Connectome SIFT2": "aparc-I SIFT2",
    "aparc a2009s MSA I Connectome Streamline Count": "aparc.a2009s-I Streamline Count",
    "Schaefer7n200p MSA I Connectome Streamline Count": "Schaefer200-I Streamline Count",
    "Schaefer7n200p MSA I Connectome SIFT2": "Schaefer200-I SIFT2",
    "aparc MSA I Connectome FA": "aparc-I FA",
    "aparc a2009s MSA I Connectome FA": "aparc.a2009s-I FA",
    "aparc MSA I Connectome Mean Length": "aparc-I Mean Length",
    "aparc a2009s MSA I Connectome SIFT2": "aparc.a2009s-I SIFT2",
    "L2 TBSS": "L2 TBSS",
    "Schaefer7n200p MSA I Connectome FA": "Schaefer200-I FA",
    "aparc a2009s MSA I Connectome Mean Length": "aparc.a2009s-I Mean Length",
    "Glasser MSA IV Connectome SIFT2": "Glasser-IV SIFT2",
    "Glasser MSA IV Connectome Streamline Count": "Glasser-IV Streamline Count",
    "L3 TBSS": "L3 TBSS",
    "ICVF TBSS": "ICVF TBSS",
    "Glasser MSA IV Connectome FA": "Glasser-IV FA",
    "Schaefer7n500p MSA IV Connectome SIFT2": "Schaefer500-IV SIFT2",
    "L1 TBSS": "L1 TBSS",
    "Schaefer7n500p MSA IV Connectome Streamline Count": "Schaefer500-IV Streamline Count",
    "FA TBSS": "FA TBSS",
    "Glasser MSA I Connectome FA": "Glasser-I FA",
    "Schaefer7n200p MSA I Connectome Mean Length": "Schaefer200-I Mean Length",
    "MD TBSS": "MD TBSS",
    "MD Probabilistic": "MD Prob.",
    "Glasser MSA I Connectome Streamline Count": "Glasser-I Streamline Count",
    "L3 Probabilistic": "L3 Prob.",
    "Schaefer7n500p MSA IV Connectome FA": "Schaefer500-IV FA",
    "Glasser MSA I Connectome SIFT2": "Glasser-I SIFT2",
    "L2 Probabilistic": "L2 Prob.",
    "L1 Probabilistic": "L1 Prob.",
    "ISOVF TBSS": "ISOVF TBSS",
    "Glasser MSA IV Connectome Mean Length": "Glasser-IV Mean Length",
    "Glasser MSA I Connectome Mean Length": "Glasser-I Mean Length",
    "OD TBSS": "OD TBSS",
    "ICVF Probabilistic": "ICVF Prob.",
    "MO TBSS": "MO TBSS",
    "ISOVF Probabilistic": "ISOVF Prob.",
    "FA Probabilistic": "FA Prob.",
    "Schaefer7n500p MSA IV Connectome Mean Length": "Schaefer500-IV Mean Length",
    "OD Probabilistic": "OD Prob.",
    "MO Probabilistic": "MO Prob."
}

dti_perf_pls["Modality"] = dti_perf_pls["Modality"].map(dti_rename_dict)

In [ ]:
dti_pls_res = pd.merge(dti_perf_pls, commonality_metrics_dti_pls, on = 'Modality')
rs_pls_res = pd.merge(rs_perf_pls, commonality_metrics_rs_pls, on = 'Modality')
struct_pls_res = pd.merge(struct_perf_pls, commonality_metrics_struct_pls, on = 'Modality')

pls_res = pd.concat([struct_pls_res, dti_pls_res, rs_pls_res], axis=0)
all_res = pd.concat([stack_res, pls_res], axis=0)

In [ ]:
corr, p_value = pearsonr(pls_res['Pearson r'], pls_res['Var Exp MRI'])
corr

In [ ]:
def get_bootstrap_ci(bootstrap_distribution, confidence_level=0.95):
    """Compute confidence intervals with input validation"""
    alpha = 1 - confidence_level
    lower = np.percentile(bootstrap_distribution, (alpha / 2) * 100).round(3)
    upper = np.percentile(bootstrap_distribution, (1 - alpha / 2) * 100).round(3)
    return (lower, upper)


def save_dataframe_to_files(df, pathname, filename):
    """
    Save a pandas DataFrame to both CSV and Excel formats in the specified location.
    Creates directories if they don't exist.
    """
    # Ensure the filename doesn't include extensions
    base_name = os.path.splitext(filename)[0]
    
    # Create directory if it doesn't exist
    os.makedirs(pathname, exist_ok=True)
    
    # Create full paths
    csv_path = os.path.join(pathname, f"{base_name}.csv")
    excel_path = os.path.join(pathname, f"{base_name}.xlsx")
    
    try:
        # Save files
        df.to_csv(csv_path, index=False)
        df.to_excel(excel_path, index=False, engine='openpyxl')
        
        print(f"Files saved successfully:\n- {csv_path}\n- {excel_path}")
    except Exception as e:
        print(f"Error saving files: {str(e)}")



def bootstrap_pearson_correlation(df, x_col='Pearson r', y_col='Var Exp MRI', n_bootstraps=5000, ci_level=0.95, random_state=None):
    """
    Compute bootstrap confidence intervals for Pearson correlation between two columns
    
    Parameters:
    df : pd.DataFrame
        Input dataframe containing the variables
    x_col, y_col : str
        Column names for the two variables
    n_bootstraps : int
        Number of bootstrap samples (default: 5000)
    ci_level : float
        Confidence level (default: 0.95)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    dict : Contains original correlation, p-value, CI, and full bootstrap distribution
    """
    if random_state is not None:
        np.random.seed(random_state)
        
    x = df[x_col].values
    y = df[y_col].values
    n = len(x)
    
    # Original correlation
    original_r, original_p = pearsonr(x, y)
    
    # Bootstrap distribution
    boot_samples = np.zeros(n_bootstraps)
    for i in range(n_bootstraps):
        idx = np.random.choice(n, n, replace=True)
        boot_samples[i] = pearsonr(x[idx], y[idx])[0]
    
    # Calculate CI using your existing function
    ci_lower, ci_upper = get_bootstrap_ci(boot_samples, confidence_level=ci_level)
    
    # Prepare results in your standard format
    results = {
        'Modality': 'Combined',  # Or specify as needed
        'Metric': 'r',
        'Mean': np.mean(boot_samples).round(3),
        'Median': np.median(boot_samples).round(3),
        'Std': np.std(boot_samples).round(3),
        'CI_lower': ci_lower,
        'CI_upper': ci_upper,
        'Original_r': original_r,
        'Original_p': original_p,
        'bootstrap_dist': boot_samples
    }
    
    return results

# Example usage with your data:
bootstrap_results = bootstrap_pearson_correlation(
    pls_res,
    x_col='Pearson r',
    y_col='Var Exp MRI',
    n_bootstraps=5000,
    ci_level=0.95,
    random_state=42
)

# Convert to DataFrame for consistency with your workflow
results_df = pd.DataFrame([{
    'Modality': bootstrap_results['Modality'],
    'Metric': bootstrap_results['Metric'],
    'Mean': bootstrap_results['Mean'],
    'Median': bootstrap_results['Median'],
    'Std': bootstrap_results['Std'],
    'CI_lower': bootstrap_results['CI_lower'],
    'CI_upper': bootstrap_results['CI_upper']
}])

# Save results using your existing function
save_dataframe_to_files(
    results_df,
    pathname='/media/hcs-sci-psy-narun/IBu/Articles-Conferences/Articles/eLife/rev1-analysis/bootstrap',
    filename='bootstrap_prediction_commonality'
)

# Print summary
print("\n=== Bootstrap Results for Pearson r ~ Var Exp MRI ===")
print(f"Original correlation: {bootstrap_results['Original_r']:.3f}")
print(f"Original p-value: {bootstrap_results['Original_p']:.4f}")
print(f"Bootstrapped 95% CI: [{bootstrap_results['CI_lower']:.3f}, {bootstrap_results['CI_upper']:.3f}]")

In [ ]:
# Scatterplot of correlation
colors = []
for modality in pls_res['Modality'].to_list():
    if modality in struct_pls_res['Modality'].to_list():
        colors.append('#DF8F44FF') #DF8F4499
    elif modality in dti_pls_res['Modality'].to_list():
        colors.append('#5C88DAFF') #00A1D599
    elif modality in rs_pls_res['Modality'].to_list():
        colors.append('#2C715FFF') #79AF9799
    else:
        colors.append('black')

#'#5C88DAFF', '#2C715FFF', '#DF8F44FF'
plt.figure(figsize=(8, 7))
plt.scatter(pls_res['Pearson r'], pls_res['Var Exp MRI'], s=70, alpha=0.7, linewidth=1, c=colors)
slope, intercept = np.polyfit(pls_res['Pearson r'], pls_res['Var Exp MRI'], 1)
reg_line = slope * pls_res['Pearson r'] + intercept
plt.plot(pls_res['Pearson r'], reg_line, color='red', label='Regression line', linewidth=0.5)
plt.xlabel('Performance of a neuroimaging phenotype\n in predicting cognition (Pearson $r$)', fontsize=18)
plt.ylabel('Proportion of the\ncognition-mental health relationship\ncaptured by a neuroimaging phenotype (%)', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.text(0.17, 22, '$r$=0.97', fontsize=23, color='black') 
plt.grid(False)


plt.savefig("/media/hcs-sci-psy-narun/IBu/Articles-Conferences/Articles/eLife/figures/Fig4a.png",
            bbox_inches ="tight", 
            pad_inches = 1, 
            transparent = False, 
            facecolor ="w", 
            edgecolor ='w', 
            orientation ='landscape',
            format='png')

plt.show()

# Demographics distribution in commonality analysis samples

## dwMRI

In [ ]:
all_g_mh_dti_demo_concat = []
for fold in folds:
    g_pred_dti_rf = pd.read_csv(f'/PLS/brain/stacking/g/DTI_All_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    g_pred_mh = g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_dti_rf, on='eid').merge(demo, on='eid')
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_dti_all_stack_test': 'g_pred_dti'})
    all_g_mh_dti_demo_concat.append(all_g)
    all_g_mh_dti_demo = pd.concat(all_g_mh_dti_demo_concat, axis=0, ignore_index=True)

## rsMRI

In [ ]:
all_g_mh_rs_demo_concat = []
for fold in folds:
    g_pred_rs_idp_rf = pd.read_csv(f'/PLS/brain/stacking/g/RS_IDP_Timeseries_best_metrics_target_pred_2nd_level_rf_test_fold_{fold}.csv')
    g_pred_mh = g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_rs_idp_rf, on='eid').merge(demo, on='eid') #.drop(columns=['eid'])
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_rs_idp_ts_best_stack_test': 'g_pred_rs_idp'})
    all_g_mh_rs_demo_concat.append(all_g)
    all_g_mh_rs_demo = pd.concat(all_g_mh_rs_demo_concat, axis=0, ignore_index=True)

## sMRI

In [ ]:
all_g_mh_t1_demo_concat = []
for fold in folds:
    g_pred_t1_svr = pd.read_csv(f'/PLS/brain/stacking/g/T1_T2_whole_brain_target_pred_2nd_level_svr_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_t1_svr, on='eid').merge(demo, on='eid') #.drop(columns=['eid'])
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_mribest_stack_test': 'g_pred_t1'}) #the column was incorrectly names g_pred_mribest_stack_test instead of g_pred_t1_stack_test
    all_g_mh_t1_demo_concat.append(all_g)
    all_g_mh_t1_demo = pd.concat(all_g_mh_t1_demo_concat, axis=0, ignore_index=True)

## All MRI

In [ ]:
all_g_mh_mri_demo_concat = []
for fold in folds:
    g_pred_mri_all = pd.read_csv(f'/PLS/brain/stacking/g/All_modalities_target_pred_2nd_level_xgb_test_fold_{fold}.csv')
    g_pred_mh = pd.concat([pd.read_csv(f'/mental_health/folds/fold_{fold}/g_pred/g_pred_mh_fold_{fold}.csv'), pd.read_csv(f'/mental_health/folds/fold_{fold}/suppl/g_test_matched_id_fold_{fold}.csv')], axis=1)
    all_g = pd.read_csv(f'/PLS/g_factor/g_test_with_id_fold_{fold}.csv').merge(g_pred_mh, on='eid').merge(g_pred_mri_all, on='eid').merge(demo, on='eid')
    all_g = all_g.rename(columns={'g': 'g_real', 'g_pred_mribest_stack_test': 'g_pred_mri'})
    all_g_mh_mri_demo_concat.append(all_g)
    all_g_mh_mri_demo = pd.concat(all_g_mh_mri_demo_concat, axis=0) #, ignore_index=True)

In [ ]:
basic_confounds_min = pd.read_csv('/ML_DATASETS/basic_confounds_min.csv')[['Sex', 'Age when attended assessment centre', 'eid']]
age_sex = pd.DataFrame(basic_confounds_min)
age_sex.columns = ['Sex', 'Age', 'eid']
commonality_dw_age_sex = all_g_mh_dti_demo.copy()
commonality_rs_age_sex = all_g_mh_rs_demo.copy()
commonality_str_age_sex = all_g_mh_t1_demo.copy()
commonality_mri_age_sex = all_g_mh_mri_demo.copy()

In [ ]:
# Check sample sizes for each modality
#dwMRI
print('dwMRI: Sample size', commonality_dw_age_sex.shape[0])
print('dwMRI: Mean age', commonality_dw_age_sex['Age'].mean().round(2))
print(f"dwMRI: SD age {commonality_dw_age_sex['Age'].std():.3f}")
print('dwMRI: Age when attended assessment centre, mean:', commonality_dw_age_sex['Age'].mean().round(2), 'SD:', f"{commonality_dw_age_sex['Age'].std():.3f}")
print('dwMRI: Age max range:', commonality_dw_age_sex['Age'].max())
print('dwMRI: Age min range:', commonality_dw_age_sex['Age'].min())
print('dwMRI: Proportion of males:', (commonality_dw_age_sex['Sex'].value_counts()[1] / len(commonality_dw_age_sex['Sex']) * 100).round(2))
print('dwMRI: Proportion of females:', (commonality_dw_age_sex['Sex'].value_counts()[0] / len(commonality_dw_age_sex['Sex']) * 100).round(2))

print('--------------------------------------------------------')
#rsMRI
print('rsMRI: Sample size', commonality_rs_age_sex.shape[0])
print('rsMRI: Mean age', commonality_rs_age_sex['Age'].mean().round(2))
print(f"rsMRI: SD age {commonality_rs_age_sex['Age'].std():.3f}")
print('rsMRI: Age when attended assessment centre, mean:', commonality_rs_age_sex['Age'].mean().round(2), 'SD:', f"{commonality_rs_age_sex['Age'].std():.3f}")
print('rsMRI: Age max range:', commonality_rs_age_sex['Age'].max())
print('rsMRI: Age min range:', commonality_rs_age_sex['Age'].min())
print('rsMRI: Proportion of males:', (commonality_rs_age_sex['Sex'].value_counts()[1] / len(commonality_rs_age_sex['Sex']) * 100).round(2))
print('rsMRI: Proportion of females:', (commonality_rs_age_sex['Sex'].value_counts()[0] / len(commonality_rs_age_sex['Sex']) * 100).round(2))

print('--------------------------------------------------------')
#sMRI
print('sMRI: Sample size', commonality_str_age_sex.shape[0])
print('sMRI: Mean age', commonality_str_age_sex['Age'].mean().round(2))
print(f"sMRI: SD age {commonality_str_age_sex['Age'].std():.3f}")
print('sMRI: Age when attended assessment centre, mean:', commonality_str_age_sex['Age'].mean().round(2), 'SD:', f"{commonality_str_age_sex['Age'].std():.3f}")
print('sMRI: Age max range:', commonality_str_age_sex['Age'].max())
print('sMRI: Age min range:', commonality_str_age_sex['Age'].min())
print('sMRI: Proportion of males:', (commonality_str_age_sex['Sex'].value_counts()[1] / len(commonality_str_age_sex['Sex']) * 100).round(2))
print('sMRI: Proportion of females:', (commonality_str_age_sex['Sex'].value_counts()[0] / len(commonality_str_age_sex['Sex']) * 100).round(2))

print('--------------------------------------------------------')
#MRI
print('MRI: Sample size', commonality_mri_age_sex.shape[0])
print('MRI: Mean age', commonality_mri_age_sex['Age'].mean().round(2))
print(f"MRI: SD age {commonality_mri_age_sex['Age'].std():.3f}")
print('MRI: Age when attended assessment centre, mean:', commonality_mri_age_sex['Age'].mean().round(2), 'SD:', f"{commonality_mri_age_sex['Age'].std():.3f}")
print('MRI: Age max range:', commonality_mri_age_sex['Age'].max())
print('MRI: Age min range:', commonality_mri_age_sex['Age'].min())
print('MRI: Proportion of males:', (commonality_mri_age_sex['Sex'].value_counts()[1] / len(commonality_mri_age_sex['Sex']) * 100).round(2))
print('MRI: Proportion of females:', (commonality_mri_age_sex['Sex'].value_counts()[0] / len(commonality_mri_age_sex['Sex']) * 100).round(2))